In [3]:
country_name = 'United_States'

In [4]:
country_name_edited = country_name.replace("_", "-")

In [5]:
# cell 02
import sagemaker
bucket=sagemaker.Session().default_bucket()
prefix = 'sagemaker/xgboost-'+country_name_edited+'-new-run-fixed-02-28-mick'
 
# Define IAM role
import boto3
import re
from sagemaker import get_execution_role

role = get_execution_role()

In [6]:
# cell 03
import numpy as np                                # For matrix operations and numerical processing
import pandas as pd                               # For munging tabular data
import matplotlib.pyplot as plt                   # For charts and visualizations
from IPython.display import Image                 # For displaying images in the notebook
from IPython.display import display               # For displaying outputs in the notebook
from time import gmtime, strftime                 # For labeling SageMaker models, endpoints, etc.
import sys                                        # For writing outputs to notebook
import math                                       # For ceiling function
import json                                       # For parsing hosting outputs
import os                                         # For manipulating filepath names
import sagemaker 
import zipfile     # Amazon SageMaker's Python SDK provides many helper functions

In [7]:
%%time 

# cell 06

# Define local file path
local_file_path = "../eda/with-wle-latent/new_PISA_cleaned_dataset.csv"  # Change as needed

# Define S3 details
bucket_name = "sagemaker-us-west-2-986030204467"
file_key = "capstone/testfiles/new_PISA_cleaned_dataset.csv"

# Check if the file exists locally
if os.path.exists(local_file_path):
    print("📂 Loading data from local file...")
    data = pd.read_csv(local_file_path, usecols=None)
    
else:
    print("☁️ Downloading data from S3...")
    
    # Create S3 client
    s3_client = boto3.client("s3")

    # Download the file from S3
    response = s3_client.get_object(Bucket=bucket_name, Key=file_key)

    # Read the file into pandas DataFrame
    data = pd.read_csv(response["Body"], usecols=None)

    # Save a local copy for future use
    data.to_csv(local_file_path, index=False)
    print(f"✅ File saved locally as {local_file_path}")

# Display first few rows
#data.head()

pd.set_option('display.max_columns', 500)     # Make sure we can see all of the columns
pd.set_option('display.max_rows', 20)         # Keep the output on one page
data

📂 Loading data from local file...
CPU times: user 52.8 s, sys: 5.23 s, total: 58 s
Wall time: 1min 2s


,CNT,CNTSCHID,CNTSTUID,MATH_Proficient,SISCO,ST250Q01JA,ST250Q02JA,ST250Q03JA,ST250Q04JA,ST250Q05JA,ST251Q01JA,ST251Q02JA,ST251Q03JA,ST251Q04JA,ST251Q06JA,ST251Q07JA,ST253Q01JA,ST254Q01JA,ST254Q02JA,ST254Q03JA,ST254Q04JA,ST254Q05JA,ST254Q06JA,ST255Q01JA,ST256Q01JA,ST256Q02JA,ST256Q03JA,ST256Q06JA,ST256Q07JA,ST256Q08JA,ST256Q09JA,ST256Q10JA,ST267Q01JA,ST267Q02JA,ST267Q03JA,ST267Q04JA,ST267Q05JA,ST267Q06JA,ST267Q07JA,ST267Q08JA,ST034Q01TA,ST034Q02TA,ST034Q03TA,ST034Q04TA,ST034Q05TA,ST034Q06TA,ST038Q03NA,ST038Q04NA,ST038Q05NA,ST038Q06NA,ST038Q07NA,ST038Q08NA,ST038Q09JA,ST038Q10JA,ST038Q11JA,ST265Q01JA,ST265Q02JA,ST265Q03JA,ST265Q04JA,ST266Q01JA,ST266Q02JA,ST266Q03JA,ST266Q04JA,ST266Q05JA,ST307Q01JA,ST307Q02JA,ST307Q03JA,ST307Q04JA,ST307Q05JA,ST307Q06JA,ST307Q07JA,ST307Q08JA,ST307Q09JA,ST307Q10JA,ST301Q01JA,ST301Q02JA,ST301Q03JA,ST301Q04JA,ST301Q05JA,ST301Q06JA,ST301Q07JA,ST301Q08JA,ST301Q09JA,ST301Q10JA,ST343Q01JA,ST343Q02JA,ST343Q03JA,ST343Q04JA,ST343Q05JA,ST343Q06JA,ST343Q07JA,ST343Q08JA,ST343Q09JA,ST343Q10JA,ST311Q01JA,ST311Q02JA,ST311Q03JA,ST311Q04JA,ST311Q05JA,ST311Q06JA,ST311Q07JA,ST311Q08JA,ST311Q09JA,ST311Q10JA,ST305Q01JA,ST305Q02JA,ST305Q03JA,ST305Q04JA,ST305Q05JA,ST305Q06JA,ST305Q07JA,ST305Q08JA,ST305Q09JA,ST305Q10JA,ST345Q01JA,ST345Q02JA,ST345Q03JA,ST345Q04JA,ST345Q05JA,ST345Q06JA,ST345Q07JA,ST345Q08JA,ST345Q09JA,ST345Q10JA,ST313Q01JA,ST313Q02JA,ST313Q03JA,ST313Q04JA,ST313Q05JA,ST313Q06JA,ST313Q07JA,ST313Q08JA,ST313Q09JA,ST313Q10JA,ST263Q02JA,ST263Q04JA,ST263Q06JA,ST263Q08JA,ST273Q01JA,ST273Q02JA,ST273Q03JA,ST273Q04JA,ST273Q05JA,ST273Q06JA,ST273Q07JA,ST270Q01JA,ST270Q02JA,ST270Q03JA,ST270Q04JA,ST285Q01JA,ST285Q02JA,ST285Q03JA,ST285Q04JA,ST285Q05JA,ST285Q06JA,ST285Q07JA,ST285Q08JA,ST285Q09JA,ST283Q01JA,ST283Q02JA,ST283Q03JA,ST283Q04JA,ST283Q05JA,ST283Q06JA,ST283Q07JA,ST283Q08JA,ST283Q09JA,ST275Q01WA,ST275Q02WA,ST275Q03WA,ST275Q04WA,ST275Q05WA,ST275Q06WA,ST275Q07WA,ST275Q08WA,ST275Q09WA,ST276Q01JA,ST276Q02JA,ST276Q03JA,ST276Q04JA,ST276Q05JA,ST276Q06JA,ST276Q07JA,ST276Q08JA,ST276Q09JA,ST276Q10JA,ST290Q01WA,ST290Q02WA,ST290Q03WA,ST290Q04WA,ST290Q05WA,ST290Q06WA,ST290Q07WA,ST290Q08WA,ST290Q09WA,ST291Q01JA,ST291Q02JA,ST291Q03JA,ST291Q04JA,ST291Q05JA,ST291Q06JA,ST291Q07JA,ST291Q08JA,ST291Q09JA,ST291Q10JA,ST289Q01WA,ST289Q02JA,ST289Q04JA,ST289Q05WA,ST289Q06JA,ST289Q07JA,ST289Q08WA,ST289Q09WA,ST289Q10WA,ST289Q14JA,ST293Q01JA,ST293Q02JA,ST293Q03JA,ST293Q05JA,ST293Q06JA,ST293Q07JA,ST293Q08JA,ST293Q09JA,ST292Q01JA,ST292Q02JA,ST292Q03JA,ST292Q04JA,ST292Q05JA,ST292Q06JA,ST334Q01JA,ST334Q02JA,ST334Q03JA,ST334Q04JA,ST334Q05JA,ST334Q06JA,ST334Q07JA,ST334Q08JA,ST334Q09JA,ST334Q10JA,ST335Q01JA,ST335Q02JA,ST335Q03JA,ST335Q05JA,ST335Q06JA,ST335Q07JA,ST336Q01JA,ST336Q03JA,ST336Q04JA,ST336Q05JA,ST336Q06JA,...,ST349Q01JA_2,ST349Q01JA_3,ST349Q01JA_4,ST349Q01JA_0,LANGN_105,LANGN_108,LANGN_118,LANGN_140,LANGN_148,LANGN_150,LANGN_156,LANGN_200,LANGN_204,LANGN_232,LANGN_273,LANGN_313,LANGN_316,LANGN_322,LANGN_329,LANGN_344,LANGN_351,LANGN_415,LANGN_463,LANGN_493,LANGN_496,LANGN_500,LANGN_520,LANGN_531,LANGN_602,LANGN_606,LANGN_615,LANGN_621,LANGN_625,LANGN_640,LANGN_641,LANGN_663,LANGN_669,LANGN_670,LANGN_800,LANGN_801,LANGN_802,LANGN_804,LANGN_805,LANGN_806,LANGN_807,LANGN_808,LANGN_865,LANGN_892,LANGN_895,LANGN_917,SC177Q01JA_1,SC177Q01JA_2,SC177Q01JA_3,SC177Q02JA_1,SC177Q02JA_2,SC177Q02JA_3,SC177Q03JA_1,SC177Q03JA_2,SC177Q03JA_3,MATHEXC_0,MATHEXC_1,MATHEXC_2,MATHEXC_3,SCHLTYPE_1,SCHLTYPE_2,SCHLTYPE_3,LANGN_121,LANGN_130,LANGN_137,LANGN_170,LANGN_244,LANGN_258,LANGN_263,LANGN_264,LANGN_266,LANGN_317,LANGN_340,LANGN_369,LANGN_381,LANGN_404,LANGN_420,LANGN_449,LANGN_467,LANGN_494,LANGN_495,LANGN_514,LANGN_523,LANGN_529,LANGN_540,LANGN_547,LANGN_600,LANGN_607,LANGN_618,LANGN_619,LANGN_630,LANGN_635,LANGN_650,LANGN_661,LANGN_673,LANGN_674,LANGN_809,LANGN_810,LANGN_811,LANGN_812,LANGN_813,LANGN_814,LANGN_815,LANGN_816,LANGN_818,LANGN_832,LANGN_868,LANGN_870,LANGN_920,LANGN_921,LANGN_113,LANGN_147,LANGN_275,LANGN_286,LANGN_363,LANGN_422,LANGN_434,LANG

In [9]:
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder, StandardScaler

import tensorflow as tf
from tensorflow.keras import layers, models

###############################################
# 1) LOAD DATA
###############################################
#data_path = "/content/gdrive/MyDrive/Colab Notebooks/Untitled folder/new_PISA_cleaned_dataset.csv"  # Update if needed

#if not os.path.exists(data_path):
    #print("❌ File not found. Please upload 'PISA_cleaned_dataset.csv' into your environment.")
    #raise SystemExit

#data = pd.read_csv(data_path)
print("Data loaded. Full shape:", data.shape)
display(data.head())

###############################################
# 2) MAKE A COPY OF THE DATA FOR MODELING
###############################################
model_data = data.copy()
print("\nInitial model_data shape:", model_data.shape)

###############################################
# 3) DROP UNWANTED COLUMNS
###############################################
columns_to_remove = [
    "CNTSCHID", "CNTSTUID", "OECD",
    "HOMEPOS", "RELATST", "BELONG", "BULLIED", "FEELSAFE", "SCHRISK", "PERSEVAGR",
    "CURIOAGR", "COOPAGR", "EMPATAGR", "ASSERAGR", "STRESAGR", "EMOCOAGR", "GROSAGR",
    "INFOSEEK", "FAMSUP", "DISCLIM", "TEACHSUP", "COGACRCO", "COGACMCO", "EXPOFA",
    "EXPO21ST", "MATHEFF", "MATHEF21", "FAMCON", "ANXMAT", "MATHPERS", "CREATEFF",
    "CREATSCH", "CREATFAM", "CREATAS", "CREATOOS", "CREATOP", "OPENART", "IMAGINE",
    "SCHSUST", "LEARRES", "PROBSELF", "FAMSUPSL", "FEELLAH", "SDLEFF", "ICTRES",
    "FLSCHOOL", "FLMULTSB", "FLFAMILY", "ACCESSFP", "FLCONFIN", "FLCONICT", "ACCESSFA",
    "ATTCONFM", "FRINFLFM", "ICTSCH", "ICTHOME", "ICTQUAL", "ICTSUBJ", "ICTENQ",
    "ICTFEED", "ICTOUT", "ICTWKDY", "ICTWKEND", "ICTREG", "ICTINFO", "ICTEFFIC",
    "BODYIMA", "SOCONPA", "LIFESAT", "PSYCHSYM", "SOCCON", "EXPWB", "CURSUPP",
    "PQMIMP", "PQMCAR", "PARINVOL", "PQSCHOOL", "PASCHPOL", "ATTIMMP", "CREATHME",
    "CREATACT", "CREATOPN", "CREATOR", "SCHAUTO", "TCHPART", "EDULEAD", "INSTLEAD",
    "ENCOURPG", "DIGDVPOL", "TEAFDBK", "MTTRAIN", "DMCVIEWS", "NEGSCLIM", "STAFFSHORT",
    "EDUSHORT", "STUBEHA", "TEACHBEHA", "STDTEST", "TDTEST", "ALLACTIV", "BCREATSC",
    "CREENVSC", "ACTCRESC", "OPENCUL", "PROBSCRI", "SCPREPBP", "SCPREPAP", "DIGPREP",
    "ESCS", "BMMJ1", "BFMJ2", "EFFORT1", "EFFORT2", "Option_UH",
    "SC209Q04JA", "SC209Q05JA", "SC209Q06JA"
]
model_data.drop(columns=columns_to_remove, inplace=True, errors='ignore')
print("After dropping specified columns:", model_data.shape)

###############################################
# 4) REMOVE FULLY EMPTY (ALL-NaN) COLUMNS
###############################################
all_nan_cols = model_data.columns[model_data.isnull().all()]
if len(all_nan_cols) > 0:
    print("Dropping all-NaN columns:", list(all_nan_cols))
    model_data.drop(columns=all_nan_cols, inplace=True)

###############################################
# 5) DETECT & DROP ZERO-VARIANCE COLUMNS
###############################################
numeric_cols_all = model_data.select_dtypes(include=[np.number]).columns
zero_var_cols = []
for col in numeric_cols_all:
    # If a column has <= 1 unique value (including NaN), it's effectively constant
    if model_data[col].nunique(dropna=False) <= 1:
        zero_var_cols.append(col)

if len(zero_var_cols) > 0:
    print("Dropping zero-variance columns:", zero_var_cols)
    model_data.drop(columns=zero_var_cols, inplace=True, errors='ignore')

###############################################
# 6) CHECK FOR AND HANDLE INFINITE VALUES
###############################################
num_df = model_data.select_dtypes(include=[np.number])
inf_cols = num_df.columns[np.isinf(num_df).any()]
if len(inf_cols) > 0:
    print("Found inf/-inf in columns:", list(inf_cols))
    # Replace inf/-inf with NaN to be imputed later
    for c in inf_cols:
        model_data[c] = np.where(np.isinf(model_data[c]), np.nan, model_data[c])

###############################################
# 7) REORDER COLUMNS (TARGET FIRST)
###############################################
target_col = "MATH_Proficient"
if target_col not in model_data.columns:
    raise ValueError(f"Target column '{target_col}' not found in data.")

other_cols = [c for c in model_data.columns if c != target_col]
model_data = model_data[[target_col] + other_cols]

###############################################
# 8) TRAIN/VAL/TEST SPLIT
###############################################
X = model_data.drop(columns=[target_col])
y = model_data[target_col].values

# Ensure we have at least two classes
if len(np.unique(y)) < 2:
    raise ValueError("Target column has only one unique class, cannot train a binary classifier.")

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.30, random_state=1729, shuffle=True
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, random_state=1729, shuffle=True
)

print("\nData Splits:")
print("Train shape:", X_train.shape, y_train.shape)
print("Validation shape:", X_val.shape, y_val.shape)
print("Test shape:", X_test.shape, y_test.shape)

###############################################
# 9) LABEL ENCODE OBJECT COLUMNS
###############################################
obj_cols = X_train.select_dtypes(include=['object']).columns
print("\nLabel-encoding these columns:", list(obj_cols))
for col in obj_cols:
    le = LabelEncoder()
    X_train[col] = le.fit_transform(X_train[col].astype(str))
    X_val[col]   = le.transform(X_val[col].astype(str))
    X_test[col]  = le.transform(X_test[col].astype(str))

###############################################
# 10) FILL NUMERIC MISSING WITH TRAIN MEAN
###############################################
num_cols = X_train.select_dtypes(include=[np.number]).columns
for col in num_cols:
    train_mean = X_train[col].mean()
    X_train[col].fillna(train_mean, inplace=True)
    X_val[col].fillna(train_mean, inplace=True)
    X_test[col].fillna(train_mean, inplace=True)

print("\nNaNs remaining in X_train:", X_train.isnull().sum().sum())
print("NaNs remaining in X_val:", X_val.isnull().sum().sum())
print("NaNs remaining in X_test:", X_test.isnull().sum().sum())

###############################################
# 11) STANDARD SCALING (Numeric Columns)
###############################################
scaler = StandardScaler()
X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
X_val[num_cols]   = scaler.transform(X_val[num_cols])
X_test[num_cols]  = scaler.transform(X_test[num_cols])

# Double-check no new NaNs introduced
print("NaNs in X_train after scaling:", X_train.isnull().sum().sum())
print("NaNs in X_val after scaling:", X_val.isnull().sum().sum())
print("NaNs in X_test after scaling:", X_test.isnull().sum().sum())

###############################################
# 12) BUILD THE MODEL
###############################################
tf.keras.backend.clear_session()

model = models.Sequential([
    layers.Input(shape=(X_train.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss='binary_crossentropy',
    metrics=[tf.keras.metrics.AUC(name='auc')]
)

# EarlyStopping to avoid overfitting
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_auc',
    patience=5,
    mode='max',
    restore_best_weights=True
)

###############################################
# 13) TRAIN THE MODEL
###############################################
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

###############################################
# 14) EVALUATE ON VALIDATION
###############################################
val_preds = model.predict(X_val).ravel()
print("\nAny NaNs in val_preds?", np.isnan(val_preds).any())
if np.isnan(val_preds).any():
    print("❌ NaNs present in validation predictions. Consider lowering the learning rate or checking outliers.")
else:
    val_auc = roc_auc_score(y_val, val_preds)
    print("Validation AUC:", round(val_auc, 4))

###############################################
# 15) EVALUATE ON TEST SET (IF NO NaNs)
###############################################
if not np.isnan(val_preds).any():
    test_preds = model.predict(X_test).ravel()
    print("Any NaNs in test_preds?", np.isnan(test_preds).any())
    if not np.isnan(test_preds).any():
        test_auc = roc_auc_score(y_test, test_preds)
        print("Test AUC:", round(test_auc, 4))
    else:
        print("❌ NaNs in test_preds. Check for numeric instability.")

print("\n✅ Finished end-to-end training (global model)!")


2025-03-05 02:46:02.680500: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Data loaded. Full shape: (591857, 1121)


,CNT,CNTSCHID,CNTSTUID,MATH_Proficient,SISCO,ST250Q01JA,ST250Q02JA,ST250Q03JA,ST250Q04JA,ST250Q05JA,ST251Q01JA,ST251Q02JA,ST251Q03JA,ST251Q04JA,ST251Q06JA,ST251Q07JA,ST253Q01JA,ST254Q01JA,ST254Q02JA,ST254Q03JA,ST254Q04JA,ST254Q05JA,ST254Q06JA,ST255Q01JA,ST256Q01JA,ST256Q02JA,ST256Q03JA,ST256Q06JA,ST256Q07JA,ST256Q08JA,ST256Q09JA,ST256Q10JA,ST267Q01JA,ST267Q02JA,ST267Q03JA,ST267Q04JA,ST267Q05JA,ST267Q06JA,ST267Q07JA,ST267Q08JA,ST034Q01TA,ST034Q02TA,ST034Q03TA,ST034Q04TA,ST034Q05TA,ST034Q06TA,ST038Q03NA,ST038Q04NA,ST038Q05NA,ST038Q06NA,ST038Q07NA,ST038Q08NA,ST038Q09JA,ST038Q10JA,ST038Q11JA,ST265Q01JA,ST265Q02JA,ST265Q03JA,ST265Q04JA,ST266Q01JA,ST266Q02JA,ST266Q03JA,ST266Q04JA,ST266Q05JA,ST307Q01JA,ST307Q02JA,ST307Q03JA,ST307Q04JA,ST307Q05JA,ST307Q06JA,ST307Q07JA,ST307Q08JA,ST307Q09JA,ST307Q10JA,ST301Q01JA,ST301Q02JA,ST301Q03JA,ST301Q04JA,ST301Q05JA,ST301Q06JA,ST301Q07JA,ST301Q08JA,ST301Q09JA,ST301Q10JA,ST343Q01JA,ST343Q02JA,ST343Q03JA,ST343Q04JA,ST343Q05JA,ST343Q06JA,ST343Q07JA,ST343Q08JA,ST343Q09JA,ST343Q10JA,ST311Q01JA,ST311Q02JA,ST311Q03JA,ST311Q04JA,ST311Q05JA,ST311Q06JA,ST311Q07JA,ST311Q08JA,ST311Q09JA,ST311Q10JA,ST305Q01JA,ST305Q02JA,ST305Q03JA,ST305Q04JA,ST305Q05JA,ST305Q06JA,ST305Q07JA,ST305Q08JA,ST305Q09JA,ST305Q10JA,ST345Q01JA,ST345Q02JA,ST345Q03JA,ST345Q04JA,ST345Q05JA,ST345Q06JA,ST345Q07JA,ST345Q08JA,ST345Q09JA,ST345Q10JA,ST313Q01JA,ST313Q02JA,ST313Q03JA,ST313Q04JA,ST313Q05JA,ST313Q06JA,ST313Q07JA,ST313Q08JA,ST313Q09JA,ST313Q10JA,ST263Q02JA,ST263Q04JA,ST263Q06JA,ST263Q08JA,ST273Q01JA,ST273Q02JA,ST273Q03JA,ST273Q04JA,ST273Q05JA,ST273Q06JA,ST273Q07JA,ST270Q01JA,ST270Q02JA,ST270Q03JA,ST270Q04JA,ST285Q01JA,ST285Q02JA,ST285Q03JA,ST285Q04JA,ST285Q05JA,ST285Q06JA,ST285Q07JA,ST285Q08JA,ST285Q09JA,ST283Q01JA,ST283Q02JA,ST283Q03JA,ST283Q04JA,ST283Q05JA,ST283Q06JA,ST283Q07JA,ST283Q08JA,ST283Q09JA,ST275Q01WA,ST275Q02WA,ST275Q03WA,ST275Q04WA,ST275Q05WA,ST275Q06WA,ST275Q07WA,ST275Q08WA,ST275Q09WA,ST276Q01JA,ST276Q02JA,ST276Q03JA,ST276Q04JA,ST276Q05JA,ST276Q06JA,ST276Q07JA,ST276Q08JA,ST276Q09JA,ST276Q10JA,ST290Q01WA,ST290Q02WA,ST290Q03WA,ST290Q04WA,ST290Q05WA,ST290Q06WA,ST290Q07WA,ST290Q08WA,ST290Q09WA,ST291Q01JA,ST291Q02JA,ST291Q03JA,ST291Q04JA,ST291Q05JA,ST291Q06JA,ST291Q07JA,ST291Q08JA,ST291Q09JA,ST291Q10JA,ST289Q01WA,ST289Q02JA,ST289Q04JA,ST289Q05WA,ST289Q06JA,ST289Q07JA,ST289Q08WA,ST289Q09WA,ST289Q10WA,ST289Q14JA,ST293Q01JA,ST293Q02JA,ST293Q03JA,ST293Q05JA,ST293Q06JA,ST293Q07JA,ST293Q08JA,ST293Q09JA,ST292Q01JA,ST292Q02JA,ST292Q03JA,ST292Q04JA,ST292Q05JA,ST292Q06JA,ST334Q01JA,ST334Q02JA,ST334Q03JA,ST334Q04JA,ST334Q05JA,ST334Q06JA,ST334Q07JA,ST334Q08JA,ST334Q09JA,ST334Q10JA,ST335Q01JA,ST335Q02JA,ST335Q03JA,ST335Q05JA,ST335Q06JA,ST335Q07JA,ST336Q01JA,ST336Q03JA,ST336Q04JA,ST336Q05JA,ST336Q06JA,...,ST349Q01JA_2,ST349Q01JA_3,ST349Q01JA_4,ST349Q01JA_0,LANGN_105,LANGN_108,LANGN_118,LANGN_140,LANGN_148,LANGN_150,LANGN_156,LANGN_200,LANGN_204,LANGN_232,LANGN_273,LANGN_313,LANGN_316,LANGN_322,LANGN_329,LANGN_344,LANGN_351,LANGN_415,LANGN_463,LANGN_493,LANGN_496,LANGN_500,LANGN_520,LANGN_531,LANGN_602,LANGN_606,LANGN_615,LANGN_621,LANGN_625,LANGN_640,LANGN_641,LANGN_663,LANGN_669,LANGN_670,LANGN_800,LANGN_801,LANGN_802,LANGN_804,LANGN_805,LANGN_806,LANGN_807,LANGN_808,LANGN_865,LANGN_892,LANGN_895,LANGN_917,SC177Q01JA_1,SC177Q01JA_2,SC177Q01JA_3,SC177Q02JA_1,SC177Q02JA_2,SC177Q02JA_3,SC177Q03JA_1,SC177Q03JA_2,SC177Q03JA_3,MATHEXC_0,MATHEXC_1,MATHEXC_2,MATHEXC_3,SCHLTYPE_1,SCHLTYPE_2,SCHLTYPE_3,LANGN_121,LANGN_130,LANGN_137,LANGN_170,LANGN_244,LANGN_258,LANGN_263,LANGN_264,LANGN_266,LANGN_317,LANGN_340,LANGN_369,LANGN_381,LANGN_404,LANGN_420,LANGN_449,LANGN_467,LANGN_494,LANGN_495,LANGN_514,LANGN_523,LANGN_529,LANGN_540,LANGN_547,LANGN_600,LANGN_607,LANGN_618,LANGN_619,LANGN_630,LANGN_635,LANGN_650,LANGN_661,LANGN_673,LANGN_674,LANGN_809,LANGN_810,LANGN_811,LANGN_812,LANGN_813,LANGN_814,LANGN_815,LANGN_816,LANGN_818,LANGN_832,LANGN_868,LANGN_870,LANGN_920,LANGN_921,LANGN_113,LANGN_147,LANGN_275,LANGN_286,LANGN_363,LANGN_422,LANGN_434,LANG


Initial model_data shape: (591857, 1121)
After dropping specified columns: (591857, 1084)

Data Splits:
Train shape: (414299, 1083) (414299,)
Validation shape: (88779, 1083) (88779,)
Test shape: (88779, 1083) (88779,)

Label-encoding these columns: ['CNT']


/tmp/ipykernel_644/3107515968.py:139: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_train[col].fillna(train_mean, inplace=True)
/tmp/ipykernel_644/3107515968.py:140: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try u


NaNs remaining in X_train: 0
NaNs remaining in X_val: 0
NaNs remaining in X_test: 0
NaNs in X_train after scaling: 0
NaNs in X_val after scaling: 0
NaNs in X_test after scaling: 0
Epoch 1/50
12947/12947 ━━━━━━━━━━━━━━━━━━━━ 28s 2ms/step - auc: 0.9137 - loss: 0.3684 - val_auc: 0.9284 - val_loss: 0.3390
Epoch 2/50
12947/12947 ━━━━━━━━━━━━━━━━━━━━ 26s 2ms/step - auc: 0.9324 - loss: 0.3286 - val_auc: 0.9302 - val_loss: 0.3341
Epoch 3/50
12947/12947 ━━━━━━━━━━━━━━━━━━━━ 26s 2ms/step - auc: 0.9379 - loss: 0.3152 - val_auc: 0.9302 - val_loss: 0.3346
Epoch 4/50
12947/12947 ━━━━━━━━━━━━━━━━━━━━ 26s 2ms/step - auc: 0.9421 - loss: 0.3049 - val_auc: 0.9300 - val_loss: 0.3366
Epoch 5/50
12947/12947 ━━━━━━━━━━━━━━━━━━━━ 26s 2ms/step - auc: 0.9460 - loss: 0.2945 - val_auc: 0.9281 - val_loss: 0.3421
Epoch 6/50
12947/12947 ━━━━━━━━━━━━━━━━━━━━ 26s 2ms/step - auc: 0.9496 - loss: 0.2846 - val_auc: 0.9278 - val_loss: 0.3465
Epoch 7/50
12947/12947 ━━━━━━━━━━━━━━━━━━━━ 26s 2ms/step - auc: 0.9533 - loss: 0.

In [10]:
###############################################
# 15) EVALUATE ON TEST SET (IF NO NaNs)
###############################################
if not np.isnan(val_preds).any():
    test_preds = model.predict(X_test).ravel()
    print("Any NaNs in test_preds?", np.isnan(test_preds).any())
    if not np.isnan(test_preds).any():
        test_auc = roc_auc_score(y_test, test_preds)
        print("Test AUC:", round(test_auc, 4))

        # ======================
        # CONFUSION MATRIX CODE
        # ======================

        # 1. Create a copy of X_test for confusion matrix analysis
        test_data_small = X_test.copy()
        # Insert the actual label into the DataFrame
        test_data_small['MATH_Proficient'] = y_test

        # 2. Assign the predictions and threshold
        predictions_small = test_preds
        threshold = 0.66  # Adjust as desired

        # 3. Create confusion matrix via crosstab
        cm_small = pd.crosstab(
            index=test_data_small['MATH_Proficient'],
            columns=np.round((predictions_small >= threshold).astype(int)),
            rownames=['actuals'],
            colnames=['predictions']
        )

        # 4. Extract TN, FP, FN, TP
        TN_small = cm_small.loc[0.0, 0.0]
        FP_small = cm_small.loc[0.0, 1.0]
        FN_small = cm_small.loc[1.0, 0.0]
        TP_small = cm_small.loc[1.0, 1.0]

        # 5. Calculate various metrics
        accuracy_small = (TP_small + TN_small) / (TP_small + TN_small + FP_small + FN_small) * 100
        precision_small = (TP_small / (TP_small + FP_small) * 100) if (TP_small + FP_small) > 0 else 0
        recall_small = (TP_small / (TP_small + FN_small) * 100) if (TP_small + FN_small) > 0 else 0
        f1_score_small = (2 * (precision_small * recall_small) / (precision_small + recall_small)
                          if (precision_small + recall_small) > 0 else 0)
        specificity_small = (TN_small / (TN_small + FP_small) * 100) if (TN_small + FP_small) > 0 else 0

        # 6. Print results
        print("\nConfusion Matrix:\n", cm_small)
        print("\nAccuracy (20 features): {:.1f}".format(accuracy_small))
        print("F1 Score (20 features): {:.1f}".format(f1_score_small))
        print("Precision (20 features): {:.1f}".format(precision_small))
        print("Recall (20 features): {:.1f}".format(recall_small))
        print("Specificity (20 features): {:.1f}".format(specificity_small))

    else:
        print("❌ NaNs in test_preds. Check for numeric instability.")

print("\n✅ Finished end-to-end training!")

2775/2775 ━━━━━━━━━━━━━━━━━━━━ 2s 850us/step
Any NaNs in test_preds? False
Test AUC: 0.9301

Confusion Matrix:
 predictions      0      1
actuals                  
0.0          37162   3411
1.0          11554  36652

Accuracy (20 features): 83.1
F1 Score (20 features): 83.0
Precision (20 features): 91.5
Recall (20 features): 76.0
Specificity (20 features): 91.6

✅ Finished end-to-end training!


In [15]:
# 1) Install shap if not installed (done once in your environment)
# !pip install shap

import shap
import numpy as np

# 2) We already have our trained model, X_train, X_test, etc.

# 3) Prepare background data
X_train_np = X_train.values if hasattr(X_train, "values") else X_train
background_size = 100
background_indices = np.random.choice(X_train_np.shape[0], background_size, replace=False)
background_data = X_train_np[background_indices]

# 4) Initialize DeepExplainer
explainer = shap.DeepExplainer(model, background_data)

# 5) Choose the data to explain (test set)
X_test_np = X_test.values if hasattr(X_test, "values") else X_test
shap_values = explainer.shap_values(X_test_np)


/opt/conda/lib/python3.11/site-packages/shap/explainers/_deep/deep_tf.py:99: UserWarning: Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.
  warnings.warn("Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.")
/opt/conda/lib/python3.11/site-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: keras_tensor
Received: inputs=['Tensor(shape=(100, 1083))']
  warnings.warn(msg)
/opt/conda/lib/python3.11/site-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: keras_tensor
Received: inputs=['Tensor(shape=(200, 1083))']
  warnings.warn(msg)
/opt/conda/lib/python3.11/site-packages/keras/src/models/functi

In [16]:
import numpy as np
import pandas as pd

# 1) Ensure shap_values is a NumPy array of shape (n_samples, n_features).
#    For a single-output model, shap.DeepExplainer often returns a list with one array:
if isinstance(shap_values, list) and len(shap_values) == 1:
    shap_values_arr = shap_values[0]
else:
    shap_values_arr = shap_values

# 2) Compute mean absolute SHAP values per feature
mean_abs_shap = np.mean(np.abs(shap_values_arr), axis=0)

# 3) Retrieve feature names (if X_test is a DataFrame)
if isinstance(X_test, pd.DataFrame):
    feature_names = X_test.columns
else:
    # Fallback names if X_test is a NumPy array
    feature_names = [f"Feature_{i}" for i in range(shap_values_arr.shape[1])]

# 4) Sort features by their mean absolute SHAP value (descending order)
# Reshape mean_abs_shap to 1D for proper indexing
mean_abs_shap = mean_abs_shap.reshape(-1)  # This is the important change

sorted_idx = np.argsort(mean_abs_shap)[::-1]
top_n = 20  # how many top features to show
top_idx = sorted_idx[:top_n]

# 5) Create a DataFrame for easier display
top_features_df = pd.DataFrame({
    'Feature': [feature_names[i] for i in top_idx],
    'MeanAbsSHAP': mean_abs_shap[top_idx]
})

# 6) Print or display
print("Top 20 features by mean absolute SHAP value:")
display(top_features_df)

Top 20 features by mean absolute SHAP value:


,Feature,MeanAbsSHAP
0,ST253Q01JA,0.032830
1,ST004D01T,0.026750
2,ST059Q02JA,0.023766
3,ST255Q01JA,0.019007
4,LANGN_156,0.016472
5,ST349Q01JA_1,0.016402
6,GRADE,0.014917
7,ST297Q09JA,0.013518
8,ST349Q01JA_2,0.013166
9,ST268Q04JA,0.013036


In [17]:
data.shape

(591857, 1121)

In [18]:
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder, StandardScaler

import tensorflow as tf
from tensorflow.keras import layers, models

###############################################
# 1) LOAD DATA
###############################################
print("Data loaded. Full shape:", data.shape)
display(data.head())

###############################################
# 2) MAKE A COPY OF THE DATA FOR MODELING
###############################################
model_data = data.copy()
print("\nInitial model_data shape:", model_data.shape)

###############################################
# 3) DROP UNWANTED COLUMNS
###############################################
columns_to_remove = [
    "CNTSCHID", "CNTSTUID", "OECD",
    "HOMEPOS", "RELATST", "BELONG", "BULLIED", "FEELSAFE", "SCHRISK", "PERSEVAGR",
    "CURIOAGR", "COOPAGR", "EMPATAGR", "ASSERAGR", "STRESAGR", "EMOCOAGR", "GROSAGR",
    "INFOSEEK", "FAMSUP", "DISCLIM", "TEACHSUP", "COGACRCO", "COGACMCO", "EXPOFA",
    "EXPO21ST", "MATHEFF", "MATHEF21", "FAMCON", "ANXMAT", "MATHPERS", "CREATEFF",
    "CREATSCH", "CREATFAM", "CREATAS", "CREATOOS", "CREATOP", "OPENART", "IMAGINE",
    "SCHSUST", "LEARRES", "PROBSELF", "FAMSUPSL", "FEELLAH", "SDLEFF", "ICTRES",
    "FLSCHOOL", "FLMULTSB", "FLFAMILY", "ACCESSFP", "FLCONFIN", "FLCONICT", "ACCESSFA",
    "ATTCONFM", "FRINFLFM", "ICTSCH", "ICTHOME", "ICTQUAL", "ICTSUBJ", "ICTENQ",
    "ICTFEED", "ICTOUT", "ICTWKDY", "ICTWKEND", "ICTREG", "ICTINFO", "ICTEFFIC",
    "BODYIMA", "SOCONPA", "LIFESAT", "PSYCHSYM", "SOCCON", "EXPWB", "CURSUPP",
    "PQMIMP", "PQMCAR", "PARINVOL", "PQSCHOOL", "PASCHPOL", "ATTIMMP", "CREATHME",
    "CREATACT", "CREATOPN", "CREATOR", "SCHAUTO", "TCHPART", "EDULEAD", "INSTLEAD",
    "ENCOURPG", "DIGDVPOL", "TEAFDBK", "MTTRAIN", "DMCVIEWS", "NEGSCLIM", "STAFFSHORT",
    "EDUSHORT", "STUBEHA", "TEACHBEHA", "STDTEST", "TDTEST", "ALLACTIV", "BCREATSC",
    "CREENVSC", "ACTCRESC", "OPENCUL", "PROBSCRI", "SCPREPBP", "SCPREPAP", "DIGPREP",
    "ESCS", "BMMJ1", "BFMJ2", "EFFORT1", "EFFORT2", "Option_UH",
    "SC209Q04JA", "SC209Q05JA", "SC209Q06JA"
]
model_data.drop(columns=columns_to_remove, inplace=True, errors='ignore')
print("After dropping specified columns:", model_data.shape)

###############################################
# 3.5) LIMIT TO YOUR TOP FEATURES + TARGET
###############################################
target_col = "MATH_Proficient"

# Make sure this list matches exactly your column names in model_data:
features_to_keep = [
    "ST253Q01JA", "ST004D01T", "ST059Q02JA", "ST255Q01JA", "LANGN_156",
    "GRADE", "ST349Q01JA_1", "SC211Q03JA", "ST259Q01JA", "ST256Q03JA",
    "ST268Q04JA", "REPEAT", "ST297Q09JA", "WORKPAY", "ST251Q04JA",
    "EXERPRAC", "EXPECEDU", "ST349Q01JA_2", "ST251Q06JA", "CNT"
]

# Subset only target + these features
needed_cols = [target_col] + features_to_keep
model_data = model_data[needed_cols]
print("\nColumns retained:", list(model_data.columns))
print("Shape after subsetting:", model_data.shape)

###############################################
# 4) REMOVE FULLY EMPTY (ALL-NaN) COLUMNS
###############################################
all_nan_cols = model_data.columns[model_data.isnull().all()]
if len(all_nan_cols) > 0:
    print("Dropping all-NaN columns:", list(all_nan_cols))
    model_data.drop(columns=all_nan_cols, inplace=True)

###############################################
# 5) DETECT & DROP ZERO-VARIANCE COLUMNS
###############################################
numeric_cols_all = model_data.select_dtypes(include=[np.number]).columns
zero_var_cols = []
for col in numeric_cols_all:
    # If a column has <= 1 unique value (including NaN), it's effectively constant
    if model_data[col].nunique(dropna=False) <= 1:
        zero_var_cols.append(col)

if len(zero_var_cols) > 0:
    print("Dropping zero-variance columns:", zero_var_cols)
    model_data.drop(columns=zero_var_cols, inplace=True, errors='ignore')

###############################################
# 6) CHECK FOR AND HANDLE INFINITE VALUES
###############################################
num_df = model_data.select_dtypes(include=[np.number])
inf_cols = num_df.columns[np.isinf(num_df).any()]
if len(inf_cols) > 0:
    print("Found inf/-inf in columns:", list(inf_cols))
    # Replace inf/-inf with NaN to be imputed later
    for c in inf_cols:
        model_data[c] = np.where(np.isinf(model_data[c]), np.nan, model_data[c])

###############################################
# 7) REORDER COLUMNS (TARGET FIRST) -- optional
###############################################
# If not already first, do it explicitly:
if model_data.columns[0] != target_col:
    other_cols = [c for c in model_data.columns if c != target_col]
    model_data = model_data[[target_col] + other_cols]

###############################################
# 8) TRAIN/VAL/TEST SPLIT
###############################################
X = model_data.drop(columns=[target_col])
y = model_data[target_col].values

# Ensure we have at least two classes
if len(np.unique(y)) < 2:
    raise ValueError("Target column has only one unique class, cannot train a binary classifier.")

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.30, random_state=1729, shuffle=True
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, random_state=1729, shuffle=True
)

print("\nData Splits:")
print("Train shape:", X_train.shape, y_train.shape)
print("Validation shape:", X_val.shape, y_val.shape)
print("Test shape:", X_test.shape, y_test.shape)

###############################################
# 9) LABEL ENCODE OBJECT COLUMNS
###############################################
obj_cols = X_train.select_dtypes(include=['object']).columns
print("\nLabel-encoding these columns:", list(obj_cols))
for col in obj_cols:
    le = LabelEncoder()
    X_train[col] = le.fit_transform(X_train[col].astype(str))
    X_val[col]   = le.transform(X_val[col].astype(str))
    X_test[col]  = le.transform(X_test[col].astype(str))

###############################################
# 10) FILL NUMERIC MISSING WITH TRAIN MEAN
###############################################
num_cols = X_train.select_dtypes(include=[np.number]).columns
for col in num_cols:
    train_mean = X_train[col].mean()
    X_train[col].fillna(train_mean, inplace=True)
    X_val[col].fillna(train_mean, inplace=True)
    X_test[col].fillna(train_mean, inplace=True)

print("\nNaNs remaining in X_train:", X_train.isnull().sum().sum())
print("NaNs remaining in X_val:", X_val.isnull().sum().sum())
print("NaNs remaining in X_test:", X_test.isnull().sum().sum())

###############################################
# 11) STANDARD SCALING (Numeric Columns)
###############################################
scaler = StandardScaler()
X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
X_val[num_cols]   = scaler.transform(X_val[num_cols])
X_test[num_cols]  = scaler.transform(X_test[num_cols])

# Double-check no new NaNs introduced
print("NaNs in X_train after scaling:", X_train.isnull().sum().sum())
print("NaNs in X_val after scaling:", X_val.isnull().sum().sum())
print("NaNs in X_test after scaling:", X_test.isnull().sum().sum())

###############################################
# 12) BUILD THE MODEL
###############################################
tf.keras.backend.clear_session()

model = models.Sequential([
    layers.Input(shape=(X_train.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss='binary_crossentropy',
    metrics=[tf.keras.metrics.AUC(name='auc')]
)

# EarlyStopping to avoid overfitting
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_auc',
    patience=5,
    mode='max',
    restore_best_weights=True
)

###############################################
# 13) TRAIN THE MODEL
###############################################
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

###############################################
# 14) EVALUATE ON VALIDATION
###############################################
val_preds = model.predict(X_val).ravel()
print("\nAny NaNs in val_preds?", np.isnan(val_preds).any())
if np.isnan(val_preds).any():
    print("❌ NaNs present in validation predictions. Consider lowering the learning rate or checking outliers.")
else:
    val_auc = roc_auc_score(y_val, val_preds)
    print("Validation AUC:", round(val_auc, 4))

###############################################
# 15) EVALUATE ON TEST SET (IF NO NaNs)
###############################################
if not np.isnan(val_preds).any():
    test_preds = model.predict(X_test).ravel()
    print("Any NaNs in test_preds?", np.isnan(test_preds).any())
    if not np.isnan(test_preds).any():
        test_auc = roc_auc_score(y_test, test_preds)
        print("Test AUC:", round(test_auc, 4))
    else:
        print("❌ NaNs in test_preds. Check for numeric instability.")

print("\n✅ Finished end-to-end training with top features only!")


Data loaded. Full shape: (591857, 1121)


,CNT,CNTSCHID,CNTSTUID,MATH_Proficient,SISCO,ST250Q01JA,ST250Q02JA,ST250Q03JA,ST250Q04JA,ST250Q05JA,ST251Q01JA,ST251Q02JA,ST251Q03JA,ST251Q04JA,ST251Q06JA,ST251Q07JA,ST253Q01JA,ST254Q01JA,ST254Q02JA,ST254Q03JA,ST254Q04JA,ST254Q05JA,ST254Q06JA,ST255Q01JA,ST256Q01JA,ST256Q02JA,ST256Q03JA,ST256Q06JA,ST256Q07JA,ST256Q08JA,ST256Q09JA,ST256Q10JA,ST267Q01JA,ST267Q02JA,ST267Q03JA,ST267Q04JA,ST267Q05JA,ST267Q06JA,ST267Q07JA,ST267Q08JA,ST034Q01TA,ST034Q02TA,ST034Q03TA,ST034Q04TA,ST034Q05TA,ST034Q06TA,ST038Q03NA,ST038Q04NA,ST038Q05NA,ST038Q06NA,ST038Q07NA,ST038Q08NA,ST038Q09JA,ST038Q10JA,ST038Q11JA,ST265Q01JA,ST265Q02JA,ST265Q03JA,ST265Q04JA,ST266Q01JA,ST266Q02JA,ST266Q03JA,ST266Q04JA,ST266Q05JA,ST307Q01JA,ST307Q02JA,ST307Q03JA,ST307Q04JA,ST307Q05JA,ST307Q06JA,ST307Q07JA,ST307Q08JA,ST307Q09JA,ST307Q10JA,ST301Q01JA,ST301Q02JA,ST301Q03JA,ST301Q04JA,ST301Q05JA,ST301Q06JA,ST301Q07JA,ST301Q08JA,ST301Q09JA,ST301Q10JA,ST343Q01JA,ST343Q02JA,ST343Q03JA,ST343Q04JA,ST343Q05JA,ST343Q06JA,ST343Q07JA,ST343Q08JA,ST343Q09JA,ST343Q10JA,ST311Q01JA,ST311Q02JA,ST311Q03JA,ST311Q04JA,ST311Q05JA,ST311Q06JA,ST311Q07JA,ST311Q08JA,ST311Q09JA,ST311Q10JA,ST305Q01JA,ST305Q02JA,ST305Q03JA,ST305Q04JA,ST305Q05JA,ST305Q06JA,ST305Q07JA,ST305Q08JA,ST305Q09JA,ST305Q10JA,ST345Q01JA,ST345Q02JA,ST345Q03JA,ST345Q04JA,ST345Q05JA,ST345Q06JA,ST345Q07JA,ST345Q08JA,ST345Q09JA,ST345Q10JA,ST313Q01JA,ST313Q02JA,ST313Q03JA,ST313Q04JA,ST313Q05JA,ST313Q06JA,ST313Q07JA,ST313Q08JA,ST313Q09JA,ST313Q10JA,ST263Q02JA,ST263Q04JA,ST263Q06JA,ST263Q08JA,ST273Q01JA,ST273Q02JA,ST273Q03JA,ST273Q04JA,ST273Q05JA,ST273Q06JA,ST273Q07JA,ST270Q01JA,ST270Q02JA,ST270Q03JA,ST270Q04JA,ST285Q01JA,ST285Q02JA,ST285Q03JA,ST285Q04JA,ST285Q05JA,ST285Q06JA,ST285Q07JA,ST285Q08JA,ST285Q09JA,ST283Q01JA,ST283Q02JA,ST283Q03JA,ST283Q04JA,ST283Q05JA,ST283Q06JA,ST283Q07JA,ST283Q08JA,ST283Q09JA,ST275Q01WA,ST275Q02WA,ST275Q03WA,ST275Q04WA,ST275Q05WA,ST275Q06WA,ST275Q07WA,ST275Q08WA,ST275Q09WA,ST276Q01JA,ST276Q02JA,ST276Q03JA,ST276Q04JA,ST276Q05JA,ST276Q06JA,ST276Q07JA,ST276Q08JA,ST276Q09JA,ST276Q10JA,ST290Q01WA,ST290Q02WA,ST290Q03WA,ST290Q04WA,ST290Q05WA,ST290Q06WA,ST290Q07WA,ST290Q08WA,ST290Q09WA,ST291Q01JA,ST291Q02JA,ST291Q03JA,ST291Q04JA,ST291Q05JA,ST291Q06JA,ST291Q07JA,ST291Q08JA,ST291Q09JA,ST291Q10JA,ST289Q01WA,ST289Q02JA,ST289Q04JA,ST289Q05WA,ST289Q06JA,ST289Q07JA,ST289Q08WA,ST289Q09WA,ST289Q10WA,ST289Q14JA,ST293Q01JA,ST293Q02JA,ST293Q03JA,ST293Q05JA,ST293Q06JA,ST293Q07JA,ST293Q08JA,ST293Q09JA,ST292Q01JA,ST292Q02JA,ST292Q03JA,ST292Q04JA,ST292Q05JA,ST292Q06JA,ST334Q01JA,ST334Q02JA,ST334Q03JA,ST334Q04JA,ST334Q05JA,ST334Q06JA,ST334Q07JA,ST334Q08JA,ST334Q09JA,ST334Q10JA,ST335Q01JA,ST335Q02JA,ST335Q03JA,ST335Q05JA,ST335Q06JA,ST335Q07JA,ST336Q01JA,ST336Q03JA,ST336Q04JA,ST336Q05JA,ST336Q06JA,...,ST349Q01JA_2,ST349Q01JA_3,ST349Q01JA_4,ST349Q01JA_0,LANGN_105,LANGN_108,LANGN_118,LANGN_140,LANGN_148,LANGN_150,LANGN_156,LANGN_200,LANGN_204,LANGN_232,LANGN_273,LANGN_313,LANGN_316,LANGN_322,LANGN_329,LANGN_344,LANGN_351,LANGN_415,LANGN_463,LANGN_493,LANGN_496,LANGN_500,LANGN_520,LANGN_531,LANGN_602,LANGN_606,LANGN_615,LANGN_621,LANGN_625,LANGN_640,LANGN_641,LANGN_663,LANGN_669,LANGN_670,LANGN_800,LANGN_801,LANGN_802,LANGN_804,LANGN_805,LANGN_806,LANGN_807,LANGN_808,LANGN_865,LANGN_892,LANGN_895,LANGN_917,SC177Q01JA_1,SC177Q01JA_2,SC177Q01JA_3,SC177Q02JA_1,SC177Q02JA_2,SC177Q02JA_3,SC177Q03JA_1,SC177Q03JA_2,SC177Q03JA_3,MATHEXC_0,MATHEXC_1,MATHEXC_2,MATHEXC_3,SCHLTYPE_1,SCHLTYPE_2,SCHLTYPE_3,LANGN_121,LANGN_130,LANGN_137,LANGN_170,LANGN_244,LANGN_258,LANGN_263,LANGN_264,LANGN_266,LANGN_317,LANGN_340,LANGN_369,LANGN_381,LANGN_404,LANGN_420,LANGN_449,LANGN_467,LANGN_494,LANGN_495,LANGN_514,LANGN_523,LANGN_529,LANGN_540,LANGN_547,LANGN_600,LANGN_607,LANGN_618,LANGN_619,LANGN_630,LANGN_635,LANGN_650,LANGN_661,LANGN_673,LANGN_674,LANGN_809,LANGN_810,LANGN_811,LANGN_812,LANGN_813,LANGN_814,LANGN_815,LANGN_816,LANGN_818,LANGN_832,LANGN_868,LANGN_870,LANGN_920,LANGN_921,LANGN_113,LANGN_147,LANGN_275,LANGN_286,LANGN_363,LANGN_422,LANGN_434,LANG


Initial model_data shape: (591857, 1121)
After dropping specified columns: (591857, 1084)

Columns retained: ['MATH_Proficient', 'ST253Q01JA', 'ST004D01T', 'ST059Q02JA', 'ST255Q01JA', 'LANGN_156', 'GRADE', 'ST349Q01JA_1', 'SC211Q03JA', 'ST259Q01JA', 'ST256Q03JA', 'ST268Q04JA', 'REPEAT', 'ST297Q09JA', 'WORKPAY', 'ST251Q04JA', 'EXERPRAC', 'EXPECEDU', 'ST349Q01JA_2', 'ST251Q06JA', 'CNT']
Shape after subsetting: (591857, 21)

Data Splits:
Train shape: (414299, 20) (414299,)
Validation shape: (88779, 20) (88779,)
Test shape: (88779, 20) (88779,)

Label-encoding these columns: ['CNT']


/tmp/ipykernel_644/1229917044.py:149: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_train[col].fillna(train_mean, inplace=True)
/tmp/ipykernel_644/1229917044.py:150: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try u


NaNs remaining in X_train: 0
NaNs remaining in X_val: 0
NaNs remaining in X_test: 0
NaNs in X_train after scaling: 0
NaNs in X_val after scaling: 0
NaNs in X_test after scaling: 0
Epoch 1/50
12947/12947 ━━━━━━━━━━━━━━━━━━━━ 20s 1ms/step - auc: 0.8616 - loss: 0.4587 - val_auc: 0.8765 - val_loss: 0.4396
Epoch 2/50
12947/12947 ━━━━━━━━━━━━━━━━━━━━ 19s 1ms/step - auc: 0.8747 - loss: 0.4398 - val_auc: 0.8789 - val_loss: 0.4338
Epoch 3/50
12947/12947 ━━━━━━━━━━━━━━━━━━━━ 19s 1ms/step - auc: 0.8776 - loss: 0.4349 - val_auc: 0.8801 - val_loss: 0.4317
Epoch 4/50
12947/12947 ━━━━━━━━━━━━━━━━━━━━ 19s 1ms/step - auc: 0.8796 - loss: 0.4316 - val_auc: 0.8811 - val_loss: 0.4304
Epoch 5/50
12947/12947 ━━━━━━━━━━━━━━━━━━━━ 19s 1ms/step - auc: 0.8801 - loss: 0.4310 - val_auc: 0.8819 - val_loss: 0.4290
Epoch 6/50
12947/12947 ━━━━━━━━━━━━━━━━━━━━ 19s 1ms/step - auc: 0.8815 - loss: 0.4285 - val_auc: 0.8829 - val_loss: 0.4285
Epoch 7/50
12947/12947 ━━━━━━━━━━━━━━━━━━━━ 19s 1ms/step - auc: 0.8824 - loss: 0.

In [19]:
###############################################
# 15) EVALUATE ON TEST SET (IF NO NaNs)
###############################################
if not np.isnan(val_preds).any():
    test_preds = model.predict(X_test).ravel()
    print("Any NaNs in test_preds?", np.isnan(test_preds).any())
    if not np.isnan(test_preds).any():
        test_auc = roc_auc_score(y_test, test_preds)
        print("Test AUC:", round(test_auc, 4))

        # ======================
        # CONFUSION MATRIX CODE
        # ======================

        # 1. Create a copy of X_test for confusion matrix analysis
        test_data_small = X_test.copy()
        # Insert the actual label into the DataFrame
        test_data_small['MATH_Proficient'] = y_test

        # 2. Assign the predictions and threshold
        predictions_small = test_preds
        threshold = 0.66  # Adjust as desired

        # 3. Create confusion matrix via crosstab
        cm_small = pd.crosstab(
            index=test_data_small['MATH_Proficient'],
            columns=np.round((predictions_small >= threshold).astype(int)),
            rownames=['actuals'],
            colnames=['predictions']
        )

        # 4. Extract TN, FP, FN, TP
        TN_small = cm_small.loc[0.0, 0.0]
        FP_small = cm_small.loc[0.0, 1.0]
        FN_small = cm_small.loc[1.0, 0.0]
        TP_small = cm_small.loc[1.0, 1.0]

        # 5. Calculate various metrics
        accuracy_small = (TP_small + TN_small) / (TP_small + TN_small + FP_small + FN_small) * 100
        precision_small = (TP_small / (TP_small + FP_small) * 100) if (TP_small + FP_small) > 0 else 0
        recall_small = (TP_small / (TP_small + FN_small) * 100) if (TP_small + FN_small) > 0 else 0
        f1_score_small = (2 * (precision_small * recall_small) / (precision_small + recall_small)
                          if (precision_small + recall_small) > 0 else 0)
        specificity_small = (TN_small / (TN_small + FP_small) * 100) if (TN_small + FP_small) > 0 else 0

        # 6. Print results
        print("\nConfusion Matrix:\n", cm_small)
        print("\nAccuracy (20 features): {:.1f}".format(accuracy_small))
        print("F1 Score (20 features): {:.1f}".format(f1_score_small))
        print("Precision (20 features): {:.1f}".format(precision_small))
        print("Recall (20 features): {:.1f}".format(recall_small))
        print("Specificity (20 features): {:.1f}".format(specificity_small))

    else:
        print("❌ NaNs in test_preds. Check for numeric instability.")

print("\n✅ Finished end-to-end training!")

2775/2775 ━━━━━━━━━━━━━━━━━━━━ 2s 626us/step
Any NaNs in test_preds? False
Test AUC: 0.8859

Confusion Matrix:
 predictions      0      1
actuals                  
0.0          35161   5412
1.0          13694  34512

Accuracy (20 features): 78.5
F1 Score (20 features): 78.3
Precision (20 features): 86.4
Recall (20 features): 71.6
Specificity (20 features): 86.7

✅ Finished end-to-end training!


In [21]:
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder, StandardScaler

import tensorflow as tf
from tensorflow.keras import layers, models, regularizers

# ==================================================
# 1) LOAD DATA
# ==================================================
print("Data loaded. Full shape:", data.shape)
display(data.head())

# ==================================================
# 2) COPY DATA
# ==================================================
model_data = data.copy()
print("\nInitial model_data shape:", model_data.shape)

# ==================================================
# 3) DROP UNWANTED COLUMNS
# ==================================================
columns_to_remove = [
    "CNTSCHID", "CNTSTUID", "OECD",
    "HOMEPOS", "RELATST", "BELONG", "BULLIED", "FEELSAFE", "SCHRISK", "PERSEVAGR",
    "CURIOAGR", "COOPAGR", "EMPATAGR", "ASSERAGR", "STRESAGR", "EMOCOAGR", "GROSAGR",
    "INFOSEEK", "FAMSUP", "DISCLIM", "TEACHSUP", "COGACRCO", "COGACMCO", "EXPOFA",
    "EXPO21ST", "MATHEFF", "MATHEF21", "FAMCON", "ANXMAT", "MATHPERS", "CREATEFF",
    "CREATSCH", "CREATFAM", "CREATAS", "CREATOOS", "CREATOP", "OPENART", "IMAGINE",
    "SCHSUST", "LEARRES", "PROBSELF", "FAMSUPSL", "FEELLAH", "SDLEFF", "ICTRES",
    "FLSCHOOL", "FLMULTSB", "FLFAMILY", "ACCESSFP", "FLCONFIN", "FLCONICT", "ACCESSFA",
    "ATTCONFM", "FRINFLFM", "ICTSCH", "ICTHOME", "ICTQUAL", "ICTSUBJ", "ICTENQ",
    "ICTFEED", "ICTOUT", "ICTWKDY", "ICTWKEND", "ICTREG", "ICTINFO", "ICTEFFIC",
    "BODYIMA", "SOCONPA", "LIFESAT", "PSYCHSYM", "SOCCON", "EXPWB", "CURSUPP",
    "PQMIMP", "PQMCAR", "PARINVOL", "PQSCHOOL", "PASCHPOL", "ATTIMMP", "CREATHME",
    "CREATACT", "CREATOPN", "CREATOR", "SCHAUTO", "TCHPART", "EDULEAD", "INSTLEAD",
    "ENCOURPG", "DIGDVPOL", "TEAFDBK", "MTTRAIN", "DMCVIEWS", "NEGSCLIM", "STAFFSHORT",
    "EDUSHORT", "STUBEHA", "TEACHBEHA", "STDTEST", "TDTEST", "ALLACTIV", "BCREATSC",
    "CREENVSC", "ACTCRESC", "OPENCUL", "PROBSCRI", "SCPREPBP", "SCPREPAP", "DIGPREP",
    "ESCS", "BMMJ1", "BFMJ2", "EFFORT1", "EFFORT2", "Option_UH",
    "SC209Q04JA", "SC209Q05JA", "SC209Q06JA"
]
model_data.drop(columns=columns_to_remove, inplace=True, errors='ignore')
print("After dropping specified columns:", model_data.shape)

# ==================================================
# 3.5) KEEP ONLY TOP FEATURES + TARGET
# ==================================================
target_col = "MATH_Proficient"
features_to_keep = [
    "ST253Q01JA", "ST004D01T", "ST059Q02JA", "ST255Q01JA", "LANGN_156",
    "GRADE", "ST349Q01JA_1", "SC211Q03JA", "ST259Q01JA", "ST256Q03JA",
    "ST268Q04JA", "REPEAT", "ST297Q09JA", "WORKPAY", "ST251Q04JA",
    "EXERPRAC", "EXPECEDU", "ST349Q01JA_2", "ST251Q06JA", "CNT"
]
model_data = model_data[[target_col] + features_to_keep]
print("\nColumns retained:", list(model_data.columns))
print("Shape after subsetting:", model_data.shape)

# ==================================================
# 4) REMOVE ALL-NaN COLUMNS
# ==================================================
all_nan_cols = model_data.columns[model_data.isnull().all()]
if len(all_nan_cols) > 0:
    print("Dropping all-NaN columns:", list(all_nan_cols))
    model_data.drop(columns=all_nan_cols, inplace=True)

# ==================================================
# 5) DETECT & DROP ZERO-VARIANCE COLUMNS
# ==================================================
numeric_cols_all = model_data.select_dtypes(include=[np.number]).columns
zero_var_cols = []
for col in numeric_cols_all:
    if model_data[col].nunique(dropna=False) <= 1:
        zero_var_cols.append(col)

if len(zero_var_cols) > 0:
    print("Dropping zero-variance columns:", zero_var_cols)
    model_data.drop(columns=zero_var_cols, inplace=True, errors='ignore')

# ==================================================
# 6) HANDLE INFINITE VALUES
# ==================================================
num_df = model_data.select_dtypes(include=[np.number])
inf_cols = num_df.columns[np.isinf(num_df).any()]
if len(inf_cols) > 0:
    print("Found inf/-inf in columns:", list(inf_cols))
    for c in inf_cols:
        model_data[c] = np.where(np.isinf(model_data[c]), np.nan, model_data[c])

# ==================================================
# 7) TRAIN/VAL/TEST SPLIT
# ==================================================
X = model_data.drop(columns=[target_col])
y = model_data[target_col].values

if len(np.unique(y)) < 2:
    raise ValueError("Target has only one unique class, cannot train a binary classifier.")

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.30, random_state=1729, shuffle=True
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, random_state=1729, shuffle=True
)

print("\nData Splits:")
print("Train shape:", X_train.shape, y_train.shape)
print("Validation shape:", X_val.shape, y_val.shape)
print("Test shape:", X_test.shape, y_test.shape)

# ==================================================
# 8) ENCODE CATEGORICALS
# ==================================================
obj_cols = X_train.select_dtypes(include=['object']).columns
print("\nLabel-encoding these columns:", list(obj_cols))

# To avoid SettingWithCopyWarning, we can reassign columns.
for col in obj_cols:
    le = LabelEncoder()
    X_train[col] = le.fit_transform(X_train[col].astype(str))
    X_val[col]   = le.transform(X_val[col].astype(str))
    X_test[col]  = le.transform(X_test[col].astype(str))

# ==================================================
# 9) FILL NUMERIC MISSING WITH TRAIN MEAN
# ==================================================
num_cols = X_train.select_dtypes(include=[np.number]).columns
for col in num_cols:
    train_mean = X_train[col].mean()
    X_train.loc[:, col] = X_train[col].fillna(train_mean)
    X_val.loc[:, col]   = X_val[col].fillna(train_mean)
    X_test.loc[:, col]  = X_test[col].fillna(train_mean)

print("\nNaNs remaining in X_train:", X_train.isnull().sum().sum())
print("NaNs remaining in X_val:", X_val.isnull().sum().sum())
print("NaNs remaining in X_test:", X_test.isnull().sum().sum())

# ==================================================
# 10) STANDARD SCALING
# ==================================================
scaler = StandardScaler()
X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
X_val[num_cols]   = scaler.transform(X_val[num_cols])
X_test[num_cols]  = scaler.transform(X_test[num_cols])

print("NaNs in X_train after scaling:", X_train.isnull().sum().sum())
print("NaNs in X_val after scaling:", X_val.isnull().sum().sum())
print("NaNs in X_test after scaling:", X_test.isnull().sum().sum())

# ==================================================
# 11) BUILD IMPROVED MODEL
# ==================================================
tf.keras.backend.clear_session()

# Example: deeper network with BatchNorm, Dropout, and L2 regularization
model = models.Sequential([
    layers.Input(shape=(X_train.shape[1],)),
    layers.Dense(
        128, activation='relu',
        kernel_regularizer=regularizers.l2(0.0005)
    ),
    layers.BatchNormalization(),
    layers.Dropout(0.3),

    layers.Dense(
        64, activation='relu',
        kernel_regularizer=regularizers.l2(0.0005)
    ),
    layers.BatchNormalization(),
    layers.Dropout(0.3),

    layers.Dense(32, activation='relu'),

    layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss='binary_crossentropy',
    metrics=[tf.keras.metrics.AUC(name='auc')]
)

# Callback: EarlyStopping to avoid overfitting
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_auc',
    patience=5,       # stop after 5 epochs with no improvement
    mode='max',
    restore_best_weights=True
)

# (Optional) Callback: ReduceLROnPlateau to lower LR if val_auc plateaus
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_auc',
    factor=0.5,       # reduce LR by half
    patience=3,       # wait 3 epochs of no improvement
    mode='max',
    min_lr=1e-6       # don't go lower than this
)

# ==================================================
# 12) TRAIN THE MODEL
# ==================================================
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=32,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

# ==================================================
# 13) EVALUATE
# ==================================================
val_preds = model.predict(X_val).ravel()
print("\nAny NaNs in val_preds?", np.isnan(val_preds).any())
if np.isnan(val_preds).any():
    print("❌ NaNs in val_preds. Consider adjusting LR or checking outliers.")
else:
    val_auc = roc_auc_score(y_val, val_preds)
    print("Validation AUC:", round(val_auc, 4))

if not np.isnan(val_preds).any():
    test_preds = model.predict(X_test).ravel()
    print("Any NaNs in test_preds?", np.isnan(test_preds).any())
    if not np.isnan(test_preds).any():
        test_auc = roc_auc_score(y_test, test_preds)
        print("Test AUC:", round(test_auc, 4))
    else:
        print("❌ NaNs in test_preds. Check numeric stability.")

print("\n✅ Finished training with an adjusted architecture!")


Data loaded. Full shape: (591857, 1121)


,CNT,CNTSCHID,CNTSTUID,MATH_Proficient,SISCO,ST250Q01JA,ST250Q02JA,ST250Q03JA,ST250Q04JA,ST250Q05JA,ST251Q01JA,ST251Q02JA,ST251Q03JA,ST251Q04JA,ST251Q06JA,ST251Q07JA,ST253Q01JA,ST254Q01JA,ST254Q02JA,ST254Q03JA,ST254Q04JA,ST254Q05JA,ST254Q06JA,ST255Q01JA,ST256Q01JA,ST256Q02JA,ST256Q03JA,ST256Q06JA,ST256Q07JA,ST256Q08JA,ST256Q09JA,ST256Q10JA,ST267Q01JA,ST267Q02JA,ST267Q03JA,ST267Q04JA,ST267Q05JA,ST267Q06JA,ST267Q07JA,ST267Q08JA,ST034Q01TA,ST034Q02TA,ST034Q03TA,ST034Q04TA,ST034Q05TA,ST034Q06TA,ST038Q03NA,ST038Q04NA,ST038Q05NA,ST038Q06NA,ST038Q07NA,ST038Q08NA,ST038Q09JA,ST038Q10JA,ST038Q11JA,ST265Q01JA,ST265Q02JA,ST265Q03JA,ST265Q04JA,ST266Q01JA,ST266Q02JA,ST266Q03JA,ST266Q04JA,ST266Q05JA,ST307Q01JA,ST307Q02JA,ST307Q03JA,ST307Q04JA,ST307Q05JA,ST307Q06JA,ST307Q07JA,ST307Q08JA,ST307Q09JA,ST307Q10JA,ST301Q01JA,ST301Q02JA,ST301Q03JA,ST301Q04JA,ST301Q05JA,ST301Q06JA,ST301Q07JA,ST301Q08JA,ST301Q09JA,ST301Q10JA,ST343Q01JA,ST343Q02JA,ST343Q03JA,ST343Q04JA,ST343Q05JA,ST343Q06JA,ST343Q07JA,ST343Q08JA,ST343Q09JA,ST343Q10JA,ST311Q01JA,ST311Q02JA,ST311Q03JA,ST311Q04JA,ST311Q05JA,ST311Q06JA,ST311Q07JA,ST311Q08JA,ST311Q09JA,ST311Q10JA,ST305Q01JA,ST305Q02JA,ST305Q03JA,ST305Q04JA,ST305Q05JA,ST305Q06JA,ST305Q07JA,ST305Q08JA,ST305Q09JA,ST305Q10JA,ST345Q01JA,ST345Q02JA,ST345Q03JA,ST345Q04JA,ST345Q05JA,ST345Q06JA,ST345Q07JA,ST345Q08JA,ST345Q09JA,ST345Q10JA,ST313Q01JA,ST313Q02JA,ST313Q03JA,ST313Q04JA,ST313Q05JA,ST313Q06JA,ST313Q07JA,ST313Q08JA,ST313Q09JA,ST313Q10JA,ST263Q02JA,ST263Q04JA,ST263Q06JA,ST263Q08JA,ST273Q01JA,ST273Q02JA,ST273Q03JA,ST273Q04JA,ST273Q05JA,ST273Q06JA,ST273Q07JA,ST270Q01JA,ST270Q02JA,ST270Q03JA,ST270Q04JA,ST285Q01JA,ST285Q02JA,ST285Q03JA,ST285Q04JA,ST285Q05JA,ST285Q06JA,ST285Q07JA,ST285Q08JA,ST285Q09JA,ST283Q01JA,ST283Q02JA,ST283Q03JA,ST283Q04JA,ST283Q05JA,ST283Q06JA,ST283Q07JA,ST283Q08JA,ST283Q09JA,ST275Q01WA,ST275Q02WA,ST275Q03WA,ST275Q04WA,ST275Q05WA,ST275Q06WA,ST275Q07WA,ST275Q08WA,ST275Q09WA,ST276Q01JA,ST276Q02JA,ST276Q03JA,ST276Q04JA,ST276Q05JA,ST276Q06JA,ST276Q07JA,ST276Q08JA,ST276Q09JA,ST276Q10JA,ST290Q01WA,ST290Q02WA,ST290Q03WA,ST290Q04WA,ST290Q05WA,ST290Q06WA,ST290Q07WA,ST290Q08WA,ST290Q09WA,ST291Q01JA,ST291Q02JA,ST291Q03JA,ST291Q04JA,ST291Q05JA,ST291Q06JA,ST291Q07JA,ST291Q08JA,ST291Q09JA,ST291Q10JA,ST289Q01WA,ST289Q02JA,ST289Q04JA,ST289Q05WA,ST289Q06JA,ST289Q07JA,ST289Q08WA,ST289Q09WA,ST289Q10WA,ST289Q14JA,ST293Q01JA,ST293Q02JA,ST293Q03JA,ST293Q05JA,ST293Q06JA,ST293Q07JA,ST293Q08JA,ST293Q09JA,ST292Q01JA,ST292Q02JA,ST292Q03JA,ST292Q04JA,ST292Q05JA,ST292Q06JA,ST334Q01JA,ST334Q02JA,ST334Q03JA,ST334Q04JA,ST334Q05JA,ST334Q06JA,ST334Q07JA,ST334Q08JA,ST334Q09JA,ST334Q10JA,ST335Q01JA,ST335Q02JA,ST335Q03JA,ST335Q05JA,ST335Q06JA,ST335Q07JA,ST336Q01JA,ST336Q03JA,ST336Q04JA,ST336Q05JA,ST336Q06JA,...,ST349Q01JA_2,ST349Q01JA_3,ST349Q01JA_4,ST349Q01JA_0,LANGN_105,LANGN_108,LANGN_118,LANGN_140,LANGN_148,LANGN_150,LANGN_156,LANGN_200,LANGN_204,LANGN_232,LANGN_273,LANGN_313,LANGN_316,LANGN_322,LANGN_329,LANGN_344,LANGN_351,LANGN_415,LANGN_463,LANGN_493,LANGN_496,LANGN_500,LANGN_520,LANGN_531,LANGN_602,LANGN_606,LANGN_615,LANGN_621,LANGN_625,LANGN_640,LANGN_641,LANGN_663,LANGN_669,LANGN_670,LANGN_800,LANGN_801,LANGN_802,LANGN_804,LANGN_805,LANGN_806,LANGN_807,LANGN_808,LANGN_865,LANGN_892,LANGN_895,LANGN_917,SC177Q01JA_1,SC177Q01JA_2,SC177Q01JA_3,SC177Q02JA_1,SC177Q02JA_2,SC177Q02JA_3,SC177Q03JA_1,SC177Q03JA_2,SC177Q03JA_3,MATHEXC_0,MATHEXC_1,MATHEXC_2,MATHEXC_3,SCHLTYPE_1,SCHLTYPE_2,SCHLTYPE_3,LANGN_121,LANGN_130,LANGN_137,LANGN_170,LANGN_244,LANGN_258,LANGN_263,LANGN_264,LANGN_266,LANGN_317,LANGN_340,LANGN_369,LANGN_381,LANGN_404,LANGN_420,LANGN_449,LANGN_467,LANGN_494,LANGN_495,LANGN_514,LANGN_523,LANGN_529,LANGN_540,LANGN_547,LANGN_600,LANGN_607,LANGN_618,LANGN_619,LANGN_630,LANGN_635,LANGN_650,LANGN_661,LANGN_673,LANGN_674,LANGN_809,LANGN_810,LANGN_811,LANGN_812,LANGN_813,LANGN_814,LANGN_815,LANGN_816,LANGN_818,LANGN_832,LANGN_868,LANGN_870,LANGN_920,LANGN_921,LANGN_113,LANGN_147,LANGN_275,LANGN_286,LANGN_363,LANGN_422,LANGN_434,LANG


Initial model_data shape: (591857, 1121)
After dropping specified columns: (591857, 1084)

Columns retained: ['MATH_Proficient', 'ST253Q01JA', 'ST004D01T', 'ST059Q02JA', 'ST255Q01JA', 'LANGN_156', 'GRADE', 'ST349Q01JA_1', 'SC211Q03JA', 'ST259Q01JA', 'ST256Q03JA', 'ST268Q04JA', 'REPEAT', 'ST297Q09JA', 'WORKPAY', 'ST251Q04JA', 'EXERPRAC', 'EXPECEDU', 'ST349Q01JA_2', 'ST251Q06JA', 'CNT']
Shape after subsetting: (591857, 21)

Data Splits:
Train shape: (414299, 20) (414299,)
Validation shape: (88779, 20) (88779,)
Test shape: (88779, 20) (88779,)

Label-encoding these columns: ['CNT']

NaNs remaining in X_train: 0
NaNs remaining in X_val: 0
NaNs remaining in X_test: 0
NaNs in X_train after scaling: 0
NaNs in X_val after scaling: 0
NaNs in X_test after scaling: 0
Epoch 1/50
12947/12947 ━━━━━━━━━━━━━━━━━━━━ 29s 2ms/step - auc: 0.8448 - loss: 0.5230 - val_auc: 0.8740 - val_loss: 0.4557 - learning_rate: 0.0010
Epoch 2/50
12947/12947 ━━━━━━━━━━━━━━━━━━━━ 27s 2ms/step - auc: 0.8608 - loss: 0.4760

In [22]:
###############################################
# 15) EVALUATE ON TEST SET (IF NO NaNs)
###############################################
if not np.isnan(val_preds).any():
    test_preds = model.predict(X_test).ravel()
    print("Any NaNs in test_preds?", np.isnan(test_preds).any())
    if not np.isnan(test_preds).any():
        test_auc = roc_auc_score(y_test, test_preds)
        print("Test AUC:", round(test_auc, 4))

        # ======================
        # CONFUSION MATRIX CODE
        # ======================

        # 1. Create a copy of X_test for confusion matrix analysis
        test_data_small = X_test.copy()
        # Insert the actual label into the DataFrame
        test_data_small['MATH_Proficient'] = y_test

        # 2. Assign the predictions and threshold
        predictions_small = test_preds
        threshold = 0.66  # Adjust as desired

        # 3. Create confusion matrix via crosstab
        cm_small = pd.crosstab(
            index=test_data_small['MATH_Proficient'],
            columns=np.round((predictions_small >= threshold).astype(int)),
            rownames=['actuals'],
            colnames=['predictions']
        )

        # 4. Extract TN, FP, FN, TP
        TN_small = cm_small.loc[0.0, 0.0]
        FP_small = cm_small.loc[0.0, 1.0]
        FN_small = cm_small.loc[1.0, 0.0]
        TP_small = cm_small.loc[1.0, 1.0]

        # 5. Calculate various metrics
        accuracy_small = (TP_small + TN_small) / (TP_small + TN_small + FP_small + FN_small) * 100
        precision_small = (TP_small / (TP_small + FP_small) * 100) if (TP_small + FP_small) > 0 else 0
        recall_small = (TP_small / (TP_small + FN_small) * 100) if (TP_small + FN_small) > 0 else 0
        f1_score_small = (2 * (precision_small * recall_small) / (precision_small + recall_small)
                          if (precision_small + recall_small) > 0 else 0)
        specificity_small = (TN_small / (TN_small + FP_small) * 100) if (TN_small + FP_small) > 0 else 0

        # 6. Print results
        print("\nConfusion Matrix:\n", cm_small)
        print("\nAccuracy (20 features): {:.1f}".format(accuracy_small))
        print("F1 Score (20 features): {:.1f}".format(f1_score_small))
        print("Precision (20 features): {:.1f}".format(precision_small))
        print("Recall (20 features): {:.1f}".format(recall_small))
        print("Specificity (20 features): {:.1f}".format(specificity_small))

    else:
        print("❌ NaNs in test_preds. Check for numeric instability.")

print("\n✅ Finished end-to-end training!")

2775/2775 ━━━━━━━━━━━━━━━━━━━━ 2s 679us/step
Any NaNs in test_preds? False
Test AUC: 0.883

Confusion Matrix:
 predictions      0      1
actuals                  
0.0          35623   4950
1.0          15014  33192

Accuracy (20 features): 77.5
F1 Score (20 features): 76.9
Precision (20 features): 87.0
Recall (20 features): 68.9
Specificity (20 features): 87.8

✅ Finished end-to-end training!


In [23]:
!pip install keras-tuner

In [28]:
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder, StandardScaler

import tensorflow as tf
from tensorflow.keras import layers, models, regularizers

# Keras Tuner imports
import keras_tuner as kt

##############################################################################
# 1) LOAD AND PREPARE DATA
##############################################################################
print("Data loaded. Full shape:", data.shape)

model_data = data.copy()
print("\nInitial model_data shape:", model_data.shape)

##############################################################################
# 2) DROP UNWANTED COLUMNS
##############################################################################
columns_to_remove = [
    "CNTSCHID", "CNTSTUID", "OECD",
    "HOMEPOS", "RELATST", "BELONG", "BULLIED", "FEELSAFE", "SCHRISK", "PERSEVAGR",
    "CURIOAGR", "COOPAGR", "EMPATAGR", "ASSERAGR", "STRESAGR", "EMOCOAGR", "GROSAGR",
    "INFOSEEK", "FAMSUP", "DISCLIM", "TEACHSUP", "COGACRCO", "COGACMCO", "EXPOFA",
    "EXPO21ST", "MATHEFF", "MATHEF21", "FAMCON", "ANXMAT", "MATHPERS", "CREATEFF",
    "CREATSCH", "CREATFAM", "CREATAS", "CREATOOS", "CREATOP", "OPENART", "IMAGINE",
    "SCHSUST", "LEARRES", "PROBSELF", "FAMSUPSL", "FEELLAH", "SDLEFF", "ICTRES",
    "FLSCHOOL", "FLMULTSB", "FLFAMILY", "ACCESSFP", "FLCONFIN", "FLCONICT", "ACCESSFA",
    "ATTCONFM", "FRINFLFM", "ICTSCH", "ICTHOME", "ICTQUAL", "ICTSUBJ", "ICTENQ",
    "ICTFEED", "ICTOUT", "ICTWKDY", "ICTWKEND", "ICTREG", "ICTINFO", "ICTEFFIC",
    "BODYIMA", "SOCONPA", "LIFESAT", "PSYCHSYM", "SOCCON", "EXPWB", "CURSUPP",
    "PQMIMP", "PQMCAR", "PARINVOL", "PQSCHOOL", "PASCHPOL", "ATTIMMP", "CREATHME",
    "CREATACT", "CREATOPN", "CREATOR", "SCHAUTO", "TCHPART", "EDULEAD", "INSTLEAD",
    "ENCOURPG", "DIGDVPOL", "TEAFDBK", "MTTRAIN", "DMCVIEWS", "NEGSCLIM", "STAFFSHORT",
    "EDUSHORT", "STUBEHA", "TEACHBEHA", "STDTEST", "TDTEST", "ALLACTIV", "BCREATSC",
    "CREENVSC", "ACTCRESC", "OPENCUL", "PROBSCRI", "SCPREPBP", "SCPREPAP", "DIGPREP",
    "ESCS", "BMMJ1", "BFMJ2", "EFFORT1", "EFFORT2", "Option_UH",
    "SC209Q04JA", "SC209Q05JA", "SC209Q06JA"
]
model_data.drop(columns=columns_to_remove, inplace=True, errors='ignore')
print("After dropping specified columns:", model_data.shape)

##############################################################################
# 3) KEEP ONLY TOP FEATURES + TARGET
##############################################################################
target_col = "MATH_Proficient"
features_to_keep = [
    "ST253Q01JA", "ST004D01T", "ST059Q02JA", "ST255Q01JA", "LANGN_156",
    "GRADE", "ST349Q01JA_1", "SC211Q03JA", "ST259Q01JA", "ST256Q03JA",
    "ST268Q04JA", "REPEAT", "ST297Q09JA", "WORKPAY", "ST251Q04JA",
    "EXERPRAC", "EXPECEDU", "ST349Q01JA_2", "ST251Q06JA", "CNT"
]
model_data = model_data[[target_col] + features_to_keep]
print("\nColumns retained:", list(model_data.columns))
print("Shape after subsetting:", model_data.shape)

##############################################################################
# 4) REMOVE ALL-NaN COLUMNS & ZERO-VARIANCE
##############################################################################
all_nan_cols = model_data.columns[model_data.isnull().all()]
if len(all_nan_cols) > 0:
    print("Dropping all-NaN columns:", list(all_nan_cols))
    model_data.drop(columns=all_nan_cols, inplace=True)

numeric_cols_all = model_data.select_dtypes(include=[np.number]).columns
zero_var_cols = [
    c for c in numeric_cols_all
    if model_data[c].nunique(dropna=False) <= 1
]
if len(zero_var_cols) > 0:
    print("Dropping zero-variance columns:", zero_var_cols)
    model_data.drop(columns=zero_var_cols, inplace=True, errors='ignore')

##############################################################################
# 5) SPLIT DATA
##############################################################################
X = model_data.drop(columns=[target_col])
y = model_data[target_col].values

if len(np.unique(y)) < 2:
    raise ValueError("Target has only one unique class.")

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.30, random_state=1729, shuffle=True
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, random_state=1729, shuffle=True
)

print("\nData Splits:")
print("Train shape:", X_train.shape, y_train.shape)
print("Validation shape:", X_val.shape, y_val.shape)
print("Test shape:", X_test.shape, y_test.shape)

##############################################################################
# 6) ENCODE CATEGORICAL + FILL MISSING
##############################################################################
obj_cols = X_train.select_dtypes(include=['object']).columns
print("\nLabel-encoding these columns:", list(obj_cols))

for col in obj_cols:
    le = LabelEncoder()
    X_train[col] = le.fit_transform(X_train[col].astype(str))
    X_val[col]   = le.transform(X_val[col].astype(str))
    X_test[col]  = le.transform(X_test[col].astype(str))

num_cols = X_train.select_dtypes(include=[np.number]).columns
for col in num_cols:
    train_mean = X_train[col].mean()
    X_train.loc[:, col] = X_train[col].fillna(train_mean)
    X_val.loc[:, col]   = X_val[col].fillna(train_mean)
    X_test.loc[:, col]  = X_test[col].fillna(train_mean)

##############################################################################
# 7) SCALE DATA
##############################################################################
scaler = StandardScaler()
X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
X_val[num_cols]   = scaler.transform(X_val[num_cols])
X_test[num_cols]  = scaler.transform(X_test[num_cols])

##############################################################################
# 8) DEFINE A HYPERMODEL FOR KERAS TUNER
##############################################################################
def build_model(hp):
    """
    Build a Keras model for hyperparameter tuning.

    hp: a HyperParameters instance from Keras Tuner
    """
    tf.keras.backend.clear_session()

    # Select how many Dense layers
    num_layers = hp.Int('num_layers', min_value=1, max_value=3, step=1)

    # Learning rate
    lr = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    # Build the Sequential model
    model = models.Sequential()
    model.add(layers.Input(shape=(X_train.shape[1],)))

    for i in range(num_layers):
        # Number of units in each dense layer
        units = hp.Int(f'units_{i}', min_value=32, max_value=256, step=32)
        model.add(layers.Dense(
            units,
            activation='relu',
            # Optional: L2 regularization
            kernel_regularizer=regularizers.l2(
                hp.Float('l2_reg', min_value=1e-5, max_value=1e-3, sampling='log')
            )
        ))

        # Optional: dropout
        if hp.Boolean(f'dropout_{i}'):
            dropout_rate = hp.Float('dropout_rate', min_value=0.1, max_value=0.5, step=0.1)
            model.add(layers.Dropout(dropout_rate))

    # Output layer
    model.add(layers.Dense(1, activation='sigmoid'))

    # Compile
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
        loss='binary_crossentropy',
        metrics=[tf.keras.metrics.AUC(name='auc')]
    )

    return model

##############################################################################
# 9) CREATE A TUNER (RANDOM SEARCH OR HYPERBAND, ETC.)
##############################################################################
tuner = kt.RandomSearch(
    hypermodel=build_model,
    objective=kt.Objective("val_auc", direction="max"),
    max_trials=1,          # number of hyperparameter configurations to try
    executions_per_trial=1, # how many times to train each configuration
    overwrite=True,
    directory="my_tuner_dir",
    project_name="pisa_hp_tuning"
)

##############################################################################
# 10) TUNE HYPERPARAMETERS
##############################################################################
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_auc',
    patience=3,
    mode='max',
    restore_best_weights=True
)

tuner.search(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=3,     # keep epochs modest for faster tuning
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

##############################################################################
# 11) GET BEST HYPERPARAMS & BUILD FINAL MODEL
##############################################################################
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print("\nBest hyperparameters found:")
for param_name, param_value in best_hps.values.items():
    print(param_name, ":", param_value)

best_model = tuner.hypermodel.build(best_hps)

# OPTIONAL: retrain the best model more thoroughly
final_history = best_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=3,            # more epochs now that we've found good hyperparams
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

##############################################################################
# 12) PRINT THE BEST MODEL'S ARCHITECTURE
##############################################################################
print("\nBest Model Architecture:")
best_model.summary()

##############################################################################
# 13) EVALUATE ON VALIDATION & TEST
##############################################################################
val_preds = best_model.predict(X_val).ravel()
val_auc = roc_auc_score(y_val, val_preds)
print(f"\nValidation AUC: {val_auc:.4f}")

test_preds = best_model.predict(X_test).ravel()
test_auc = roc_auc_score(y_test, test_preds)
print(f"Test AUC: {test_auc:.4f}")


Trial 1 Complete [00h 01m 12s]
val_auc: 0.8731532096862793

Best val_auc So Far: 0.8731532096862793
Total elapsed time: 00h 01m 12s

Best hyperparameters found:
num_layers : 3
learning_rate : 0.01
units_0 : 192
l2_reg : 2.7902109236590983e-05
dropout_0 : True
dropout_rate : 0.1
units_1 : 32
dropout_1 : False
units_2 : 32
dropout_2 : False
Epoch 1/3
12947/12947 ━━━━━━━━━━━━━━━━━━━━ 24s 2ms/step - auc: 0.8603 - loss: 0.4710 - val_auc: 0.8712 - val_loss: 0.4663
Epoch 2/3
12947/12947 ━━━━━━━━━━━━━━━━━━━━ 23s 2ms/step - auc: 0.8662 - loss: 0.4631 - val_auc: 0.8718 - val_loss: 0.4545
Epoch 3/3
12947/12947 ━━━━━━━━━━━━━━━━━━━━ 23s 2ms/step - auc: 0.8666 - loss: 0.4615 - val_auc: 0.8725 - val_loss: 0.4531

Best Model Architecture:


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 192)            │         4,032 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 192)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         6,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 33,893 (132.40 KB)

 Trainable params: 11,297 (44.13 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 22,596 (88.27 KB)

2775/2775 ━━━━━━━━━━━━━━━━━━━━ 2s 682us/step

Validation AUC: 0.8725
2775/2775 ━━━━━━━━━━━━━━━━━━━━ 2s 678us/step
Test AUC: 0.8715


In [30]:
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder, StandardScaler

import tensorflow as tf
from tensorflow.keras import layers, models

###############################################
# 1) LOAD DATA
###############################################
print("Data loaded. Full shape:", data.shape)
display(data.head())

###############################################
# 2) MAKE A COPY OF THE DATA FOR MODELING
###############################################
model_data = data.copy()
print("\nInitial model_data shape:", model_data.shape)

###############################################
# 3) DROP UNWANTED COLUMNS
###############################################
columns_to_remove = [
    "CNTSCHID", "CNTSTUID", "OECD",
    "HOMEPOS", "RELATST", "BELONG", "BULLIED", "FEELSAFE", "SCHRISK", "PERSEVAGR",
    "CURIOAGR", "COOPAGR", "EMPATAGR", "ASSERAGR", "STRESAGR", "EMOCOAGR", "GROSAGR",
    "INFOSEEK", "FAMSUP", "DISCLIM", "TEACHSUP", "COGACRCO", "COGACMCO", "EXPOFA",
    "EXPO21ST", "MATHEFF", "MATHEF21", "FAMCON", "ANXMAT", "MATHPERS", "CREATEFF",
    "CREATSCH", "CREATFAM", "CREATAS", "CREATOOS", "CREATOP", "OPENART", "IMAGINE",
    "SCHSUST", "LEARRES", "PROBSELF", "FAMSUPSL", "FEELLAH", "SDLEFF", "ICTRES",
    "FLSCHOOL", "FLMULTSB", "FLFAMILY", "ACCESSFP", "FLCONFIN", "FLCONICT", "ACCESSFA",
    "ATTCONFM", "FRINFLFM", "ICTSCH", "ICTHOME", "ICTQUAL", "ICTSUBJ", "ICTENQ",
    "ICTFEED", "ICTOUT", "ICTWKDY", "ICTWKEND", "ICTREG", "ICTINFO", "ICTEFFIC",
    "BODYIMA", "SOCONPA", "LIFESAT", "PSYCHSYM", "SOCCON", "EXPWB", "CURSUPP",
    "PQMIMP", "PQMCAR", "PARINVOL", "PQSCHOOL", "PASCHPOL", "ATTIMMP", "CREATHME",
    "CREATACT", "CREATOPN", "CREATOR", "SCHAUTO", "TCHPART", "EDULEAD", "INSTLEAD",
    "ENCOURPG", "DIGDVPOL", "TEAFDBK", "MTTRAIN", "DMCVIEWS", "NEGSCLIM", "STAFFSHORT",
    "EDUSHORT", "STUBEHA", "TEACHBEHA", "STDTEST", "TDTEST", "ALLACTIV", "BCREATSC",
    "CREENVSC", "ACTCRESC", "OPENCUL", "PROBSCRI", "SCPREPBP", "SCPREPAP", "DIGPREP",
    "ESCS", "BMMJ1", "BFMJ2", "EFFORT1", "EFFORT2", "Option_UH",
    "SC209Q04JA", "SC209Q05JA", "SC209Q06JA"
]
model_data.drop(columns=columns_to_remove, inplace=True, errors='ignore')
print("After dropping specified columns:", model_data.shape)

###############################################
# 3.5) LIMIT TO YOUR TOP FEATURES + TARGET
###############################################
target_col = "MATH_Proficient"

# Make sure this list matches exactly your column names in model_data:
features_to_keep = [
    "ST253Q01JA", "ST004D01T", "ST059Q02JA", "ST255Q01JA", "LANGN_156",
    "GRADE", "ST349Q01JA_1", "SC211Q03JA", "ST259Q01JA", "ST256Q03JA",
    "ST268Q04JA", "REPEAT", "ST297Q09JA", "WORKPAY", "ST251Q04JA",
    "EXERPRAC", "EXPECEDU", "ST349Q01JA_2", "ST251Q06JA", "CNT"
]

# Subset only target + these features
needed_cols = [target_col] + features_to_keep
model_data = model_data[needed_cols]
print("\nColumns retained:", list(model_data.columns))
print("Shape after subsetting:", model_data.shape)

###############################################
# 4) REMOVE FULLY EMPTY (ALL-NaN) COLUMNS
###############################################
all_nan_cols = model_data.columns[model_data.isnull().all()]
if len(all_nan_cols) > 0:
    print("Dropping all-NaN columns:", list(all_nan_cols))
    model_data.drop(columns=all_nan_cols, inplace=True)

###############################################
# 5) DETECT & DROP ZERO-VARIANCE COLUMNS
###############################################
numeric_cols_all = model_data.select_dtypes(include=[np.number]).columns
zero_var_cols = []
for col in numeric_cols_all:
    # If a column has <= 1 unique value (including NaN), it's effectively constant
    if model_data[col].nunique(dropna=False) <= 1:
        zero_var_cols.append(col)

if len(zero_var_cols) > 0:
    print("Dropping zero-variance columns:", zero_var_cols)
    model_data.drop(columns=zero_var_cols, inplace=True, errors='ignore')

###############################################
# 6) CHECK FOR AND HANDLE INFINITE VALUES
###############################################
num_df = model_data.select_dtypes(include=[np.number])
inf_cols = num_df.columns[np.isinf(num_df).any()]
if len(inf_cols) > 0:
    print("Found inf/-inf in columns:", list(inf_cols))
    # Replace inf/-inf with NaN to be imputed later
    for c in inf_cols:
        model_data[c] = np.where(np.isinf(model_data[c]), np.nan, model_data[c])

###############################################
# 7) REORDER COLUMNS (TARGET FIRST) -- optional
###############################################
# If not already first, do it explicitly:
if model_data.columns[0] != target_col:
    other_cols = [c for c in model_data.columns if c != target_col]
    model_data = model_data[[target_col] + other_cols]

###############################################
# 8) TRAIN/VAL/TEST SPLIT
###############################################
X = model_data.drop(columns=[target_col])
y = model_data[target_col].values

# Ensure we have at least two classes
if len(np.unique(y)) < 2:
    raise ValueError("Target column has only one unique class, cannot train a binary classifier.")

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.30, random_state=1729, shuffle=True
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, random_state=1729, shuffle=True
)

print("\nData Splits:")
print("Train shape:", X_train.shape, y_train.shape)
print("Validation shape:", X_val.shape, y_val.shape)
print("Test shape:", X_test.shape, y_test.shape)

###############################################
# 9) LABEL ENCODE OBJECT COLUMNS
###############################################
obj_cols = X_train.select_dtypes(include=['object']).columns
print("\nLabel-encoding these columns:", list(obj_cols))
for col in obj_cols:
    le = LabelEncoder()
    X_train[col] = le.fit_transform(X_train[col].astype(str))
    X_val[col]   = le.transform(X_val[col].astype(str))
    X_test[col]  = le.transform(X_test[col].astype(str))

###############################################
# 10) FILL NUMERIC MISSING WITH TRAIN MEAN
###############################################
num_cols = X_train.select_dtypes(include=[np.number]).columns
for col in num_cols:
    train_mean = X_train[col].mean()
    X_train[col].fillna(train_mean, inplace=True)
    X_val[col].fillna(train_mean, inplace=True)
    X_test[col].fillna(train_mean, inplace=True)

print("\nNaNs remaining in X_train:", X_train.isnull().sum().sum())
print("NaNs remaining in X_val:", X_val.isnull().sum().sum())
print("NaNs remaining in X_test:", X_test.isnull().sum().sum())

###############################################
# 11) STANDARD SCALING (Numeric Columns)
###############################################
scaler = StandardScaler()
X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
X_val[num_cols]   = scaler.transform(X_val[num_cols])
X_test[num_cols]  = scaler.transform(X_test[num_cols])

# Double-check no new NaNs introduced
print("NaNs in X_train after scaling:", X_train.isnull().sum().sum())
print("NaNs in X_val after scaling:", X_val.isnull().sum().sum())
print("NaNs in X_test after scaling:", X_test.isnull().sum().sum())

###############################################
# 12) BUILD THE MODEL (With Best Hyperparams)
###############################################
tf.keras.backend.clear_session()

# Best hyperparameters found:
# num_layers : 2
# learning_rate : 0.001
# units_0 : 160
# l2_reg : 1.162911054826589e-05
# dropout_0 : False
# dropout_rate : 0.5
# units_1 : 64
# dropout_1 : True
# units_2 : 192  (ignored since num_layers=2)
# dropout_2 : False (ignored since num_layers=2)

l2_reg_value = 1.162911054826589e-05
dropout_rate = 0.5  # This will be used only where 'dropout_*' is True

model = tf.keras.Sequential()

# Input layer
model.add(layers.Input(shape=(X_train.shape[1],)))

# 1st hidden layer
model.add(layers.Dense(
    units=160,
    activation='relu',
    kernel_regularizer=tf.keras.regularizers.l2(l2_reg_value)
))
# dropout_0 = False -> no dropout here

# 2nd hidden layer
model.add(layers.Dense(
    units=64,
    activation='relu',
    kernel_regularizer=tf.keras.regularizers.l2(l2_reg_value)
))
# dropout_1 = True -> apply dropout
model.add(layers.Dropout(rate=dropout_rate))

# Output layer
model.add(layers.Dense(1, activation='sigmoid'))

# Compile model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=[tf.keras.metrics.AUC(name='auc')]
)

# EarlyStopping to avoid overfitting
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_auc',
    patience=5,
    mode='max',
    restore_best_weights=True
)

###############################################
# 13) TRAIN THE MODEL
###############################################
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=3,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

###############################################
# 14) EVALUATE ON VALIDATION
###############################################
val_preds = model.predict(X_val).ravel()
print("\nAny NaNs in val_preds?", np.isnan(val_preds).any())
if np.isnan(val_preds).any():
    print("❌ NaNs present in validation predictions. Consider lowering the learning rate or checking outliers.")
else:
    val_auc = roc_auc_score(y_val, val_preds)
    print("Validation AUC:", round(val_auc, 4))

###############################################
# 15) EVALUATE ON TEST SET (IF NO NaNs)
###############################################
if not np.isnan(val_preds).any():
    test_preds = model.predict(X_test).ravel()
    print("Any NaNs in test_preds?", np.isnan(test_preds).any())
    if not np.isnan(test_preds).any():
        test_auc = roc_auc_score(y_test, test_preds)
        print("Test AUC:", round(test_auc, 4))
    else:
        print("❌ NaNs in test_preds. Check for numeric instability.")

print("\n✅ Finished end-to-end training with top features only!")

Data loaded. Full shape: (591857, 1121)


,CNT,CNTSCHID,CNTSTUID,MATH_Proficient,SISCO,ST250Q01JA,ST250Q02JA,ST250Q03JA,ST250Q04JA,ST250Q05JA,ST251Q01JA,ST251Q02JA,ST251Q03JA,ST251Q04JA,ST251Q06JA,ST251Q07JA,ST253Q01JA,ST254Q01JA,ST254Q02JA,ST254Q03JA,ST254Q04JA,ST254Q05JA,ST254Q06JA,ST255Q01JA,ST256Q01JA,ST256Q02JA,ST256Q03JA,ST256Q06JA,ST256Q07JA,ST256Q08JA,ST256Q09JA,ST256Q10JA,ST267Q01JA,ST267Q02JA,ST267Q03JA,ST267Q04JA,ST267Q05JA,ST267Q06JA,ST267Q07JA,ST267Q08JA,ST034Q01TA,ST034Q02TA,ST034Q03TA,ST034Q04TA,ST034Q05TA,ST034Q06TA,ST038Q03NA,ST038Q04NA,ST038Q05NA,ST038Q06NA,ST038Q07NA,ST038Q08NA,ST038Q09JA,ST038Q10JA,ST038Q11JA,ST265Q01JA,ST265Q02JA,ST265Q03JA,ST265Q04JA,ST266Q01JA,ST266Q02JA,ST266Q03JA,ST266Q04JA,ST266Q05JA,ST307Q01JA,ST307Q02JA,ST307Q03JA,ST307Q04JA,ST307Q05JA,ST307Q06JA,ST307Q07JA,ST307Q08JA,ST307Q09JA,ST307Q10JA,ST301Q01JA,ST301Q02JA,ST301Q03JA,ST301Q04JA,ST301Q05JA,ST301Q06JA,ST301Q07JA,ST301Q08JA,ST301Q09JA,ST301Q10JA,ST343Q01JA,ST343Q02JA,ST343Q03JA,ST343Q04JA,ST343Q05JA,ST343Q06JA,ST343Q07JA,ST343Q08JA,ST343Q09JA,ST343Q10JA,ST311Q01JA,ST311Q02JA,ST311Q03JA,ST311Q04JA,ST311Q05JA,ST311Q06JA,ST311Q07JA,ST311Q08JA,ST311Q09JA,ST311Q10JA,ST305Q01JA,ST305Q02JA,ST305Q03JA,ST305Q04JA,ST305Q05JA,ST305Q06JA,ST305Q07JA,ST305Q08JA,ST305Q09JA,ST305Q10JA,ST345Q01JA,ST345Q02JA,ST345Q03JA,ST345Q04JA,ST345Q05JA,ST345Q06JA,ST345Q07JA,ST345Q08JA,ST345Q09JA,ST345Q10JA,ST313Q01JA,ST313Q02JA,ST313Q03JA,ST313Q04JA,ST313Q05JA,ST313Q06JA,ST313Q07JA,ST313Q08JA,ST313Q09JA,ST313Q10JA,ST263Q02JA,ST263Q04JA,ST263Q06JA,ST263Q08JA,ST273Q01JA,ST273Q02JA,ST273Q03JA,ST273Q04JA,ST273Q05JA,ST273Q06JA,ST273Q07JA,ST270Q01JA,ST270Q02JA,ST270Q03JA,ST270Q04JA,ST285Q01JA,ST285Q02JA,ST285Q03JA,ST285Q04JA,ST285Q05JA,ST285Q06JA,ST285Q07JA,ST285Q08JA,ST285Q09JA,ST283Q01JA,ST283Q02JA,ST283Q03JA,ST283Q04JA,ST283Q05JA,ST283Q06JA,ST283Q07JA,ST283Q08JA,ST283Q09JA,ST275Q01WA,ST275Q02WA,ST275Q03WA,ST275Q04WA,ST275Q05WA,ST275Q06WA,ST275Q07WA,ST275Q08WA,ST275Q09WA,ST276Q01JA,ST276Q02JA,ST276Q03JA,ST276Q04JA,ST276Q05JA,ST276Q06JA,ST276Q07JA,ST276Q08JA,ST276Q09JA,ST276Q10JA,ST290Q01WA,ST290Q02WA,ST290Q03WA,ST290Q04WA,ST290Q05WA,ST290Q06WA,ST290Q07WA,ST290Q08WA,ST290Q09WA,ST291Q01JA,ST291Q02JA,ST291Q03JA,ST291Q04JA,ST291Q05JA,ST291Q06JA,ST291Q07JA,ST291Q08JA,ST291Q09JA,ST291Q10JA,ST289Q01WA,ST289Q02JA,ST289Q04JA,ST289Q05WA,ST289Q06JA,ST289Q07JA,ST289Q08WA,ST289Q09WA,ST289Q10WA,ST289Q14JA,ST293Q01JA,ST293Q02JA,ST293Q03JA,ST293Q05JA,ST293Q06JA,ST293Q07JA,ST293Q08JA,ST293Q09JA,ST292Q01JA,ST292Q02JA,ST292Q03JA,ST292Q04JA,ST292Q05JA,ST292Q06JA,ST334Q01JA,ST334Q02JA,ST334Q03JA,ST334Q04JA,ST334Q05JA,ST334Q06JA,ST334Q07JA,ST334Q08JA,ST334Q09JA,ST334Q10JA,ST335Q01JA,ST335Q02JA,ST335Q03JA,ST335Q05JA,ST335Q06JA,ST335Q07JA,ST336Q01JA,ST336Q03JA,ST336Q04JA,ST336Q05JA,ST336Q06JA,...,ST349Q01JA_2,ST349Q01JA_3,ST349Q01JA_4,ST349Q01JA_0,LANGN_105,LANGN_108,LANGN_118,LANGN_140,LANGN_148,LANGN_150,LANGN_156,LANGN_200,LANGN_204,LANGN_232,LANGN_273,LANGN_313,LANGN_316,LANGN_322,LANGN_329,LANGN_344,LANGN_351,LANGN_415,LANGN_463,LANGN_493,LANGN_496,LANGN_500,LANGN_520,LANGN_531,LANGN_602,LANGN_606,LANGN_615,LANGN_621,LANGN_625,LANGN_640,LANGN_641,LANGN_663,LANGN_669,LANGN_670,LANGN_800,LANGN_801,LANGN_802,LANGN_804,LANGN_805,LANGN_806,LANGN_807,LANGN_808,LANGN_865,LANGN_892,LANGN_895,LANGN_917,SC177Q01JA_1,SC177Q01JA_2,SC177Q01JA_3,SC177Q02JA_1,SC177Q02JA_2,SC177Q02JA_3,SC177Q03JA_1,SC177Q03JA_2,SC177Q03JA_3,MATHEXC_0,MATHEXC_1,MATHEXC_2,MATHEXC_3,SCHLTYPE_1,SCHLTYPE_2,SCHLTYPE_3,LANGN_121,LANGN_130,LANGN_137,LANGN_170,LANGN_244,LANGN_258,LANGN_263,LANGN_264,LANGN_266,LANGN_317,LANGN_340,LANGN_369,LANGN_381,LANGN_404,LANGN_420,LANGN_449,LANGN_467,LANGN_494,LANGN_495,LANGN_514,LANGN_523,LANGN_529,LANGN_540,LANGN_547,LANGN_600,LANGN_607,LANGN_618,LANGN_619,LANGN_630,LANGN_635,LANGN_650,LANGN_661,LANGN_673,LANGN_674,LANGN_809,LANGN_810,LANGN_811,LANGN_812,LANGN_813,LANGN_814,LANGN_815,LANGN_816,LANGN_818,LANGN_832,LANGN_868,LANGN_870,LANGN_920,LANGN_921,LANGN_113,LANGN_147,LANGN_275,LANGN_286,LANGN_363,LANGN_422,LANGN_434,LANG


Initial model_data shape: (591857, 1121)
After dropping specified columns: (591857, 1084)

Columns retained: ['MATH_Proficient', 'ST253Q01JA', 'ST004D01T', 'ST059Q02JA', 'ST255Q01JA', 'LANGN_156', 'GRADE', 'ST349Q01JA_1', 'SC211Q03JA', 'ST259Q01JA', 'ST256Q03JA', 'ST268Q04JA', 'REPEAT', 'ST297Q09JA', 'WORKPAY', 'ST251Q04JA', 'EXERPRAC', 'EXPECEDU', 'ST349Q01JA_2', 'ST251Q06JA', 'CNT']
Shape after subsetting: (591857, 21)

Data Splits:
Train shape: (414299, 20) (414299,)
Validation shape: (88779, 20) (88779,)
Test shape: (88779, 20) (88779,)

Label-encoding these columns: ['CNT']


/tmp/ipykernel_644/565527225.py:149: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_train[col].fillna(train_mean, inplace=True)
/tmp/ipykernel_644/565527225.py:150: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try usi


NaNs remaining in X_train: 0
NaNs remaining in X_val: 0
NaNs remaining in X_test: 0
NaNs in X_train after scaling: 0
NaNs in X_val after scaling: 0
NaNs in X_test after scaling: 0
Epoch 1/3
12947/12947 ━━━━━━━━━━━━━━━━━━━━ 22s 2ms/step - auc: 0.8602 - loss: 0.4654 - val_auc: 0.8766 - val_loss: 0.4395
Epoch 2/3
12947/12947 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step - auc: 0.8722 - loss: 0.4477 - val_auc: 0.8797 - val_loss: 0.4350
Epoch 3/3
12947/12947 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - auc: 0.8757 - loss: 0.4428 - val_auc: 0.8804 - val_loss: 0.4339
2775/2775 ━━━━━━━━━━━━━━━━━━━━ 2s 651us/step

Any NaNs in val_preds? False
Validation AUC: 0.8804
2775/2775 ━━━━━━━━━━━━━━━━━━━━ 2s 662us/step
Any NaNs in test_preds? False
Test AUC: 0.8794

✅ Finished end-to-end training with top features only!


In [31]:
###############################################
# 15) EVALUATE ON TEST SET (IF NO NaNs)
###############################################
if not np.isnan(val_preds).any():
    test_preds = model.predict(X_test).ravel()
    print("Any NaNs in test_preds?", np.isnan(test_preds).any())
    if not np.isnan(test_preds).any():
        test_auc = roc_auc_score(y_test, test_preds)
        print("Test AUC:", round(test_auc, 4))

        # ======================
        # CONFUSION MATRIX CODE
        # ======================

        # 1. Create a copy of X_test for confusion matrix analysis
        test_data_small = X_test.copy()
        # Insert the actual label into the DataFrame
        test_data_small['MATH_Proficient'] = y_test

        # 2. Assign the predictions and threshold
        predictions_small = test_preds
        threshold = 0.66  # Adjust as desired

        # 3. Create confusion matrix via crosstab
        cm_small = pd.crosstab(
            index=test_data_small['MATH_Proficient'],
            columns=np.round((predictions_small >= threshold).astype(int)),
            rownames=['actuals'],
            colnames=['predictions']
        )

        # 4. Extract TN, FP, FN, TP
        TN_small = cm_small.loc[0.0, 0.0]
        FP_small = cm_small.loc[0.0, 1.0]
        FN_small = cm_small.loc[1.0, 0.0]
        TP_small = cm_small.loc[1.0, 1.0]

        # 5. Calculate various metrics
        accuracy_small = (TP_small + TN_small) / (TP_small + TN_small + FP_small + FN_small) * 100
        precision_small = (TP_small / (TP_small + FP_small) * 100) if (TP_small + FP_small) > 0 else 0
        recall_small = (TP_small / (TP_small + FN_small) * 100) if (TP_small + FN_small) > 0 else 0
        f1_score_small = (2 * (precision_small * recall_small) / (precision_small + recall_small)
                          if (precision_small + recall_small) > 0 else 0)
        specificity_small = (TN_small / (TN_small + FP_small) * 100) if (TN_small + FP_small) > 0 else 0

        # 6. Print results
        print("\nConfusion Matrix:\n", cm_small)
        print("\nAccuracy (20 features): {:.1f}".format(accuracy_small))
        print("F1 Score (20 features): {:.1f}".format(f1_score_small))
        print("Precision (20 features): {:.1f}".format(precision_small))
        print("Recall (20 features): {:.1f}".format(recall_small))
        print("Specificity (20 features): {:.1f}".format(specificity_small))

    else:
        print("❌ NaNs in test_preds. Check for numeric instability.")

print("\n✅ Finished end-to-end training!")

2775/2775 ━━━━━━━━━━━━━━━━━━━━ 2s 642us/step
Any NaNs in test_preds? False
Test AUC: 0.8794

Confusion Matrix:
 predictions      0      1
actuals                  
0.0          35368   5205
1.0          14921  33285

Accuracy (20 features): 77.3
F1 Score (20 features): 76.8
Precision (20 features): 86.5
Recall (20 features): 69.0
Specificity (20 features): 87.2

✅ Finished end-to-end training!
